In [147]:
%run -i ../header.py
from pylatex import Tabular, MultiColumn, NoEscape

# Export

In [148]:
def cfgnameSplit(cfgname):
    cfgcode = cfgname.split(".")
    cfgtag = "" if len(cfgcode) == 1 else ".".join(cfgcode[:-1])
    cfgcode = cfgcode[-1]
    return cfgcode, cfgtag

In [149]:
def getPDGTrans(name):
    pdg_trans = pd.read_csv("../data/{}-trans-PDG.csv".format(name), header=0, index_col=[0,1])["frac_width"]\
                  .rename("PDG")
    pdg = pd.read_csv("../data/{}-spectrum.csv".format(name), header=0, index_col=0)["PDG_width"]
    for instate in pdg_trans.index.levels[0]:
        pdg_trans.loc[(instate, slice(None))] = pdg_trans[(instate, slice(None))].values*pdg[instate]*10**6
    pdg_trans.index = pd.Index([tuple("${}$".format(code2name(e)) for e in pair) for pair in pdg_trans.index.values], name=["In", "Out"])
    return pdg_trans

In [150]:
def getDengTrans(cfgcode, cfgtag=""):
    suffix = "" if cfgtag == "" else "-"+cfgtag
    deng = pd.read_csv("../data/{}.deng2017{}.csv".format(cfgcode, suffix), header=0, index_col=[0,1])
    if sp.any(pd.isna(deng["Gtot"])):
        deng.loc[pd.isna(deng["Gtot"]), "Gtot"] = deng.loc[pd.isna(deng["Gtot"]).index.values, "Ge1"]
    if sp.any(pd.isna(deng["Gtot"])):
        deng.loc[pd.isna(deng["Gtot"]), "Gtot"] = deng.loc[pd.isna(deng["Gtot"]).index.values, "Gm1"]
    deng = deng["Gtot"].rename("Deng")
    deng *= 10**6
    deng.index = pd.Index([tuple("${}$".format(code2name(e)) for e in pair) for pair in deng.index.values], name=["In", "Out"])
    return deng

In [151]:
def getRs(particle, prefix, cfgname, suffix=""):
    data = pd.read_csv("output/{}.{}/widths{}".format(prefix, cfgname, suffix), header=0, index_col = [0, 1])
    data *= 10**6
    data["in0"] /= data["width"]
    data["in2"] /= data["width"]
    data["out0"] /= data["width"]
    data["out2"] /= data["width"]
    data.index.names = ["In", "Out"]
    
    data_sub = data.query("In == '{0}'".format(particle))\
        .rename(columns={"out0": "$r^{(0)}$", "out2": "$r^{(2)}$"})\
        .drop(columns=["in0", "in2"])
    data_above = data.query("Out == '{0}'".format(particle))\
        .rename(columns={"in0": "$r^{(0)}$", "in2": "$r^{(2)}$"})\
        .drop(columns=["out0", "out2"])
    
    data = pd.concat([data_sub, data_above])
    data.columns = ["$\Gamma$", "$r^{(0)}$", "$r^{(2)}$"]
    
    data.index = pd.Index([tuple("${}$".format(code2name(e)) for e in pair) for pair in data.index.values], name=["In", "Out"])
    return data

In [200]:
def reportRs(pname, prefix, cfgnames, suffixes=["", "E1"]): # allowed suffixes: "", "E1", "ELW"
    stack = []
    deng_stack = []
    for cfgname in cfgnames:
        cfgcode, cfgtag = cfgnameSplit(cfgname)
        parts = [getRs(pname, prefix, cfgname, suffix)\
                                    .add_prefix("" if suffix == "" else suffix+"-")\
                                for suffix in suffixes]
        parts = [part for part in parts if not part.empty]
        if len(parts) > 0:
            stack.append(pd.concat(parts, axis=1))
        deng_stack.append(getDengTrans(cfgcode, cfgtag))
    res = pd.concat(stack)
    res.insert(0, "Deng", pd.concat(deng_stack))
    res.insert(0, "PDG", getPDGTrans(cfgcode.split("-")[0]))
    res = dfsort(res, lambda a,b: cmpMultyStatesByCode(prefix, cfgcode
                                                      ,[name2code(i[1:-1]) for i in a]
                                                      ,[name2code(i[1:-1]) for i in b]))
    return res

## Widths

### $\psi(1S)$ and $\Upsilon(1S)$

In [201]:
for suff in [["", "E1"], ["E1", "ELW"]]:
    print(suff)
    res = reportRs("psi_1S", "scr", ("c-scr",), suffixes=suff)
    display(res)
    with open("export/width{}.psi_1S.c-scr.tex".format("-".join(suff)), "w") as f:
        f.write(res.to_latex(escape=False, bold_rows=True, column_format="l|l|r|r|r|r|r|r|r|r"))

['', 'E1']


,,PDG,Deng,$\Gamma$,$r^{(0)}$,$r^{(2)}$,E1-$\Gamma$,E1-$r^{(0)}$,E1-$r^{(2)}$
In,Out,,,,,,,,
$\psi(1S)$,$\eta_{c}(1S)$,1.58,2.44,2.78,1,0,2.78,1,0
$\chi_{c0}(1P)$,$\psi(1S)$,133,179,204,1,0,167,1,0
$\chi_{c1}(1P)$,$\psi(1S)$,285,319,376,0.434,0,326,0.5,0
$\chi_{c2}(1P)$,$\psi(1S)$,371,292,311,0.0552,0.696,361,0.1,0.6
$\eta_{c}(2S)$,$\psi(1S)$,nan,2.29,2.91,1,0,2.91,1,0
$\chi_{c0}(2P)$,$\psi(1S)$,nan,2.3,49.9,1,0,45,1,0
$\chi_{c1}(2P)$,$\psi(1S)$,nan,88,244,0.422,0,206,0.5,0
$\chi_{c2}(2P)$,$\psi(1S)$,nan,93,231,0.0441,0.723,279,0.1,0.6


['E1', 'ELW']


,,PDG,Deng,E1-$\Gamma$,E1-$r^{(0)}$,E1-$r^{(2)}$,ELW-$\Gamma$,ELW-$r^{(0)}$,ELW-$r^{(2)}$
In,Out,,,,,,,,
$\psi(1S)$,$\eta_{c}(1S)$,1.58,2.44,2.78,1,0,2.8,1,0
$\chi_{c0}(1P)$,$\psi(1S)$,133,179,167,1,0,187,1,0
$\chi_{c1}(1P)$,$\psi(1S)$,285,319,326,0.5,0,411,0.5,0
$\chi_{c2}(1P)$,$\psi(1S)$,371,292,361,0.1,0.6,481,0.1,0.6
$\eta_{c}(2S)$,$\psi(1S)$,nan,2.29,2.91,1,0,9.19,1,0
$\chi_{c0}(2P)$,$\psi(1S)$,nan,2.3,45,1,0,3.15,1,0
$\chi_{c1}(2P)$,$\psi(1S)$,nan,88,206,0.5,0,60.1,0.5,0
$\chi_{c2}(2P)$,$\psi(1S)$,nan,93,279,0.1,0.6,136,0.1,0.6


In [202]:
for suff in [["", "E1"], ["E1", "ELW"]]:
    print(suff)
    res = reportRs("psi_1S", "lin", ("c-lin",), suffixes=suff)
    display(res)
    with open("export/width{}.psi_1S.c-lin.tex".format("-".join(suff)), "w") as f:
        f.write(res.to_latex(escape=False, bold_rows=True, column_format="l|l|r|r|r|r|r|r|r|r"))

['', 'E1']


,,PDG,Deng,$\Gamma$,$r^{(0)}$,$r^{(2)}$,E1-$\Gamma$,E1-$r^{(0)}$,E1-$r^{(2)}$
In,Out,,,,,,,,
$\psi(1S)$,$\eta_{c}(1S)$,1.58,2.39,2.57,1,0,2.57,1,0
$\chi_{c0}(1P)$,$\psi(1S)$,133,172,195,1,0,161,1,0
$\chi_{c1}(1P)$,$\psi(1S)$,285,306,350,0.438,0,306,0.5,0
$\chi_{c2}(1P)$,$\psi(1S)$,371,284,303,0.0576,0.691,349,0.1,0.6
$\eta_{c}(2S)$,$\psi(1S)$,nan,2.64,3.03,1,0,3.03,1,0
$\chi_{c0}(2P)$,$\psi(1S)$,nan,6.1,60.4,1,0,53.5,1,0
$\chi_{c1}(2P)$,$\psi(1S)$,nan,81,259,0.426,0,220,0.5,0
$\chi_{c2}(2P)$,$\psi(1S)$,nan,93,263,0.0459,0.719,315,0.1,0.6


['E1', 'ELW']


,,PDG,Deng,E1-$\Gamma$,E1-$r^{(0)}$,E1-$r^{(2)}$,ELW-$\Gamma$,ELW-$r^{(0)}$,ELW-$r^{(2)}$
In,Out,,,,,,,,
$\psi(1S)$,$\eta_{c}(1S)$,1.58,2.39,2.57,1,0,2.58,1,0
$\chi_{c0}(1P)$,$\psi(1S)$,133,172,161,1,0,180,1,0
$\chi_{c1}(1P)$,$\psi(1S)$,285,306,306,0.5,0,380,0.5,0
$\chi_{c2}(1P)$,$\psi(1S)$,371,284,349,0.1,0.6,458,0.1,0.6
$\eta_{c}(2S)$,$\psi(1S)$,nan,2.64,3.03,1,0,8.66,1,0
$\chi_{c0}(2P)$,$\psi(1S)$,nan,6.1,53.5,1,0,1.71,1,0
$\chi_{c1}(2P)$,$\psi(1S)$,nan,81,220,0.5,0,58,0.5,0
$\chi_{c2}(2P)$,$\psi(1S)$,nan,93,315,0.1,0.6,146,0.1,0.6


In [203]:
for suff in [["", "E1"], ["E1", "ELW"]]:
    print(suff)
    res = reportRs("yps_1S", "scr", ("1P1D2D.b-scr", "nSnP.b-scr"), suffixes=suff)
    display(res)
    with open("export/width{}.yps_1S.b-scr.tex".format("-".join(suff)), "w") as f:
        f.write(res.to_latex(escape=False, bold_rows=True, column_format="l|l|r|r|r|r|r|r|r|r"))

['', 'E1']


,,PDG,Deng,$\Gamma$,$r^{(0)}$,$r^{(2)}$,E1-$\Gamma$,E1-$r^{(0)}$,E1-$r^{(2)}$
In,Out,,,,,,,,
$\chi_{b0}(1P)$,$\Upsilon(1S)$,nan,27.5,28.3,1,0,26.1,1,0
$\chi_{b1}(1P)$,$\Upsilon(1S)$,nan,31.9,33.6,0.478,0,32.1,0.5,0
$\chi_{b2}(1P)$,$\Upsilon(1S)$,nan,31.8,32.8,0.0858,0.629,34.4,0.1,0.6
$\chi_{b0}(2P)$,$\Upsilon(1S)$,nan,5.54,7.97,1,0,7.07,1,0
$\chi_{b1}(2P)$,$\Upsilon(1S)$,nan,10.8,15,0.467,0,14,0.5,0
$\chi_{b2}(2P)$,$\Upsilon(1S)$,nan,12.5,16.8,0.0786,0.644,18.1,0.1,0.6
$\chi_{b0}(3P)$,$\Upsilon(1S)$,nan,1.87,3.71,1,0,3.24,1,0
$\chi_{b1}(3P)$,$\Upsilon(1S)$,nan,6.41,9.62,0.46,0,8.84,0.5,0
$\chi_{b2}(3P)$,$\Upsilon(1S)$,nan,8.17,11.7,0.0739,0.654,12.7,0.1,0.6


['E1', 'ELW']


,,PDG,Deng,E1-$\Gamma$,E1-$r^{(0)}$,E1-$r^{(2)}$,ELW-$\Gamma$,ELW-$r^{(0)}$,ELW-$r^{(2)}$
In,Out,,,,,,,,
$\chi_{b0}(1P)$,$\Upsilon(1S)$,nan,27.5,26.1,1,0,27.9,1,0
$\chi_{b1}(1P)$,$\Upsilon(1S)$,nan,31.9,32.1,0.5,0,34.9,0.5,0
$\chi_{b2}(1P)$,$\Upsilon(1S)$,nan,31.8,34.4,0.1,0.6,37.8,0.1,0.6
$\chi_{b0}(2P)$,$\Upsilon(1S)$,nan,5.54,7.07,1,0,3.79,1,0
$\chi_{b1}(2P)$,$\Upsilon(1S)$,nan,10.8,14,0.5,0,9.79,0.5,0
$\chi_{b2}(2P)$,$\Upsilon(1S)$,nan,12.5,18.1,0.1,0.6,13.8,0.1,0.6
$\chi_{b0}(3P)$,$\Upsilon(1S)$,nan,1.87,3.24,1,0,1.01,1,0
$\chi_{b1}(3P)$,$\Upsilon(1S)$,nan,6.41,8.84,0.5,0,4.87,0.5,0
$\chi_{b2}(3P)$,$\Upsilon(1S)$,nan,8.17,12.7,0.1,0.6,8.1,0.1,0.6


### $\psi(2S)$ and $\Upsilon(2S)$

In [204]:
for suff in [["", "E1"], ["E1", "ELW"]]:
    print(suff)
    res = reportRs("psi_2S", "scr", ("c-scr",), suffixes=suff)
    display(res)
    with open("export/width{}.psi_2S.c-scr.tex".format("-".join(suff)), "w") as f:
        f.write(res.to_latex(escape=False, bold_rows=True, column_format="l|l|r|r|r|r|r|r|r|r"))

['', 'E1']


PDG  Deng  $\Gamma$  $r^{(0)}$  $r^{(2)}$  \
In              Out                                                           
$\psi(2S)$      $\eta_{c}(1S)$   1.01   7.8      10.3          1          0   
                $\chi_{c0}(1P)$  29.6    22      20.2          1          0   
                $\chi_{c1}(1P)$  28.3    45      34.9      0.528          0   
                $\chi_{c2}(1P)$    27    46      42.8      0.113      0.574   
                $\eta_{c}(2S)$  0.207  0.19     0.148          1          0   
$\chi_{c0}(2P)$ $\psi(2S)$        nan    99       116          1          0   
$\chi_{c1}(2P)$ $\psi(2S)$        nan   155       189      0.459          0   
$\chi_{c2}(2P)$ $\psi(2S)$        nan   150       159     0.0715      0.659   
$\chi_{c0}(3P)$ $\psi(2S)$        nan   9.1      72.4          1          0   
$\chi_{c1}(3P)$ $\psi(2S)$        nan    74       179      0.445          0   
$\chi_{c2}(3P)$ $\psi(2S)$        nan    76       156     0.0605      0.684   

                                 E1-$\Gamma$  E1-$r^{(0)}$  E1-$r^{(2)}$  
In              Out                                                       
$\psi(2S)$      $\eta_{c}(1S)$          10.3             1             0  
                $\chi_{c0}(1P)$         24.5             1             0  
                $\chi_{c1}(1P)$         36.8           0.5             0  
                $\chi_{c2}(1P)$           41           0.1           0.6  
                $\eta_{c}(2S)$         0.148             1             0  
$\chi_{c0}(2P)$ $\psi(2S)$               103             1             0  
$\chi_{c1}(2P)$ $\psi(2S)$               174           0.5             0  
$\chi_{c2}(2P)$ $\psi(2S)$               175           0.1           0.6  
$\chi_{c0}(3P)$ $\psi(2S)$              64.7             1             0  
$\chi_{c1}(3P)$ $\psi(2S)$               159           0.5             0  
$\chi_{c2}(3P)$ $\psi(2S)$               178           0.1           0.6

['E1', 'ELW']


PDG  Deng  E1-$\Gamma$  E1-$r^{(0)}$  \
In              Out                                                      
$\psi(2S)$      $\eta_{c}(1S)$   1.01   7.8         10.3             1   
                $\chi_{c0}(1P)$  29.6    22         24.5             1   
                $\chi_{c1}(1P)$  28.3    45         36.8           0.5   
                $\chi_{c2}(1P)$    27    46           41           0.1   
                $\eta_{c}(2S)$  0.207  0.19        0.148             1   
$\chi_{c0}(2P)$ $\psi(2S)$        nan    99          103             1   
$\chi_{c1}(2P)$ $\psi(2S)$        nan   155          174           0.5   
$\chi_{c2}(2P)$ $\psi(2S)$        nan   150          175           0.1   
$\chi_{c0}(3P)$ $\psi(2S)$        nan   9.1         64.7             1   
$\chi_{c1}(3P)$ $\psi(2S)$        nan    74          159           0.5   
$\chi_{c2}(3P)$ $\psi(2S)$        nan    76          178           0.1   

                                 E1-$r^{(2)}$  ELW-$\Gamma$  ELW-$r^{(0)}$  \
In              Out                                                          
$\psi(2S)$      $\eta_{c}(1S)$              0          4.93              1   
                $\chi_{c0}(1P)$             0          30.3              1   
                $\chi_{c1}(1P)$             0          39.9            0.5   
                $\chi_{c2}(1P)$           0.6          43.2            0.1   
                $\eta_{c}(2S)$              0         0.149              1   
$\chi_{c0}(2P)$ $\psi(2S)$                  0           117              1   
$\chi_{c1}(2P)$ $\psi(2S)$                  0           231            0.5   
$\chi_{c2}(2P)$ $\psi(2S)$                0.6           254            0.1   
$\chi_{c0}(3P)$ $\psi(2S)$                  0        0.0413              1   
$\chi_{c1}(3P)$ $\psi(2S)$                  0          41.7            0.5   
$\chi_{c2}(3P)$ $\psi(2S)$                0.6            78            0.1   

                                 ELW-$r^{(2)}$  
In              Out                             
$\psi(2S)$      $\eta_{c}(1S)$               0  
                $\chi_{c0}(1P)$              0  
                $\chi_{c1}(1P)$              0  
                $\chi_{c2}(1P)$            0.6  
                $\eta_{c}(2S)$               0  
$\chi_{c0}(2P)$ $\psi(2S)$                   0  
$\chi_{c1}(2P)$ $\psi(2S)$                   0  
$\chi_{c2}(2P)$ $\psi(2S)$                 0.6  
$\chi_{c0}(3P)$ $\psi(2S)$                   0  
$\chi_{c1}(3P)$ $\psi(2S)$                   0  
$\chi_{c2}(3P)$ $\psi(2S)$                 0.6

In [205]:
for suff in [["", "E1"], ["E1", "ELW"]]:
    print(suff)
    res = reportRs("psi_2S", "lin", ("c-lin",), suffixes=suff)
    display(res)
    with open("export/width{}.psi_2S.c-lin.tex".format("-".join(suff)), "w") as f:
        f.write(res.to_latex(escape=False, bold_rows=True, column_format="l|l|r|r|r|r|r|r|r|r"))

['', 'E1']


PDG  Deng  $\Gamma$  $r^{(0)}$  $r^{(2)}$  \
In              Out                                                           
$\psi(2S)$      $\eta_{c}(1S)$   1.01  8.08      9.63          1          0   
                $\chi_{c0}(1P)$  29.6    22        21          1          0   
                $\chi_{c1}(1P)$  28.3    42      35.9      0.528          0   
                $\chi_{c2}(1P)$    27    38      40.9      0.113      0.575   
                $\eta_{c}(2S)$  0.207  0.19     0.158          1          0   
$\chi_{c0}(2P)$ $\psi(2S)$        nan   121       141          1          0   
$\chi_{c1}(2P)$ $\psi(2S)$        nan   139       219      0.457          0   
$\chi_{c2}(2P)$ $\psi(2S)$        nan   135       183     0.0698      0.663   
$\chi_{c0}(3P)$ $\psi(2S)$        nan    17       129          1          0   
$\chi_{c1}(3P)$ $\psi(2S)$        nan    94       275      0.445          0   
$\chi_{c2}(3P)$ $\psi(2S)$        nan    97       244      0.059      0.687   

                                 E1-$\Gamma$  E1-$r^{(0)}$  E1-$r^{(2)}$  
In              Out                                                       
$\psi(2S)$      $\eta_{c}(1S)$          9.63             1             0  
                $\chi_{c0}(1P)$         25.3             1             0  
                $\chi_{c1}(1P)$         37.8           0.5             0  
                $\chi_{c2}(1P)$         39.3           0.1           0.6  
                $\eta_{c}(2S)$         0.158             1             0  
$\chi_{c0}(2P)$ $\psi(2S)$               124             1             0  
$\chi_{c1}(2P)$ $\psi(2S)$               200           0.5             0  
$\chi_{c2}(2P)$ $\psi(2S)$               203           0.1           0.6  
$\chi_{c0}(3P)$ $\psi(2S)$               115             1             0  
$\chi_{c1}(3P)$ $\psi(2S)$               244           0.5             0  
$\chi_{c2}(3P)$ $\psi(2S)$               279           0.1           0.6

['E1', 'ELW']


PDG  Deng  E1-$\Gamma$  E1-$r^{(0)}$  \
In              Out                                                      
$\psi(2S)$      $\eta_{c}(1S)$   1.01  8.08         9.63             1   
                $\chi_{c0}(1P)$  29.6    22         25.3             1   
                $\chi_{c1}(1P)$  28.3    42         37.8           0.5   
                $\chi_{c2}(1P)$    27    38         39.3           0.1   
                $\eta_{c}(2S)$  0.207  0.19        0.158             1   
$\chi_{c0}(2P)$ $\psi(2S)$        nan   121          124             1   
$\chi_{c1}(2P)$ $\psi(2S)$        nan   139          200           0.5   
$\chi_{c2}(2P)$ $\psi(2S)$        nan   135          203           0.1   
$\chi_{c0}(3P)$ $\psi(2S)$        nan    17          115             1   
$\chi_{c1}(3P)$ $\psi(2S)$        nan    94          244           0.5   
$\chi_{c2}(3P)$ $\psi(2S)$        nan    97          279           0.1   

                                 E1-$r^{(2)}$  ELW-$\Gamma$  ELW-$r^{(0)}$  \
In              Out                                                          
$\psi(2S)$      $\eta_{c}(1S)$              0           4.8              1   
                $\chi_{c0}(1P)$             0          30.7              1   
                $\chi_{c1}(1P)$             0          40.9            0.5   
                $\chi_{c2}(1P)$           0.6          41.2            0.1   
                $\eta_{c}(2S)$              0         0.158              1   
$\chi_{c0}(2P)$ $\psi(2S)$                  0           143              1   
$\chi_{c1}(2P)$ $\psi(2S)$                  0           269            0.5   
$\chi_{c2}(2P)$ $\psi(2S)$                0.6           303            0.1   
$\chi_{c0}(3P)$ $\psi(2S)$                  0         0.484              1   
$\chi_{c1}(3P)$ $\psi(2S)$                  0          38.8            0.5   
$\chi_{c2}(3P)$ $\psi(2S)$                0.6          89.6            0.1   

                                 ELW-$r^{(2)}$  
In              Out                             
$\psi(2S)$      $\eta_{c}(1S)$               0  
                $\chi_{c0}(1P)$              0  
                $\chi_{c1}(1P)$              0  
                $\chi_{c2}(1P)$            0.6  
                $\eta_{c}(2S)$               0  
$\chi_{c0}(2P)$ $\psi(2S)$                   0  
$\chi_{c1}(2P)$ $\psi(2S)$                   0  
$\chi_{c2}(2P)$ $\psi(2S)$                 0.6  
$\chi_{c0}(3P)$ $\psi(2S)$                   0  
$\chi_{c1}(3P)$ $\psi(2S)$                   0  
$\chi_{c2}(3P)$ $\psi(2S)$                 0.6

In [206]:
for suff in [["", "E1"], ["E1", "ELW"]]:
    print(suff)
    res = reportRs("yps_2S", "scr", ("1P1D2D.b-scr", "nSnP.b-scr"), suffixes=suff)
    display(res)
    with open("export/width{}.yps_2S.b-scr.tex".format("-".join(suff)), "w") as f:
        f.write(res.to_latex(escape=False, bold_rows=True, column_format="l|l|r|r|r|r|r|r|r|r"))

['', 'E1']


PDG  Deng  $\Gamma$  $r^{(0)}$  $r^{(2)}$  \
In              Out                                                          
$\Upsilon(2S)$  $\chi_{b0}(1P)$ 1.22  1.09     0.861          1          0   
                $\chi_{b1}(1P)$ 2.21  2.17       1.4      0.506          0   
                $\chi_{b2}(1P)$ 2.29  2.62      1.59      0.103      0.594   
$\chi_{b0}(2P)$ $\Upsilon(2S)$   nan  14.4        13          1          0   
$\chi_{b1}(2P)$ $\Upsilon(2S)$   nan  15.3      15.4      0.488          0   
$\chi_{b2}(2P)$ $\Upsilon(2S)$   nan  15.3      15.4     0.0921      0.616   
$\chi_{b0}(3P)$ $\Upsilon(2S)$   nan  2.55      4.54          1          0   
$\chi_{b1}(3P)$ $\Upsilon(2S)$   nan  5.63      8.18      0.479          0   
$\chi_{b2}(3P)$ $\Upsilon(2S)$   nan  6.72      9.23     0.0865      0.628   

                                 E1-$\Gamma$  E1-$r^{(0)}$  E1-$r^{(2)}$  
In              Out                                                       
$\Upsilon(2S)$  $\chi_{b0}(1P)$        0.889             1             0  
                $\chi_{b1}(1P)$         1.41           0.5             0  
                $\chi_{b2}(1P)$         1.58           0.1           0.6  
$\chi_{b0}(2P)$ $\Upsilon(2S)$          12.5             1             0  
$\chi_{b1}(2P)$ $\Upsilon(2S)$          15.1           0.5             0  
$\chi_{b2}(2P)$ $\Upsilon(2S)$          15.8           0.1           0.6  
$\chi_{b0}(3P)$ $\Upsilon(2S)$          4.23             1             0  
$\chi_{b1}(3P)$ $\Upsilon(2S)$          7.84           0.5             0  
$\chi_{b2}(3P)$ $\Upsilon(2S)$          9.65           0.1           0.6

['E1', 'ELW']


PDG  Deng  E1-$\Gamma$  E1-$r^{(0)}$  \
In              Out                                                     
$\Upsilon(2S)$  $\chi_{b0}(1P)$ 1.22  1.09        0.889             1   
                $\chi_{b1}(1P)$ 2.21  2.17         1.41           0.5   
                $\chi_{b2}(1P)$ 2.29  2.62         1.58           0.1   
$\chi_{b0}(2P)$ $\Upsilon(2S)$   nan  14.4         12.5             1   
$\chi_{b1}(2P)$ $\Upsilon(2S)$   nan  15.3         15.1           0.5   
$\chi_{b2}(2P)$ $\Upsilon(2S)$   nan  15.3         15.8           0.1   
$\chi_{b0}(3P)$ $\Upsilon(2S)$   nan  2.55         4.23             1   
$\chi_{b1}(3P)$ $\Upsilon(2S)$   nan  5.63         7.84           0.5   
$\chi_{b2}(3P)$ $\Upsilon(2S)$   nan  6.72         9.65           0.1   

                                 E1-$r^{(2)}$  ELW-$\Gamma$  ELW-$r^{(0)}$  \
In              Out                                                          
$\Upsilon(2S)$  $\chi_{b0}(1P)$             0         0.909              1   
                $\chi_{b1}(1P)$             0          1.43            0.5   
                $\chi_{b2}(1P)$           0.6          1.59            0.1   
$\chi_{b0}(2P)$ $\Upsilon(2S)$              0          13.3              1   
$\chi_{b1}(2P)$ $\Upsilon(2S)$              0          16.5            0.5   
$\chi_{b2}(2P)$ $\Upsilon(2S)$            0.6          17.7            0.1   
$\chi_{b0}(3P)$ $\Upsilon(2S)$              0          1.69              1   
$\chi_{b1}(3P)$ $\Upsilon(2S)$              0          4.52            0.5   
$\chi_{b2}(3P)$ $\Upsilon(2S)$            0.6          6.26            0.1   

                                 ELW-$r^{(2)}$  
In              Out                             
$\Upsilon(2S)$  $\chi_{b0}(1P)$              0  
                $\chi_{b1}(1P)$              0  
                $\chi_{b2}(1P)$            0.6  
$\chi_{b0}(2P)$ $\Upsilon(2S)$               0  
$\chi_{b1}(2P)$ $\Upsilon(2S)$               0  
$\chi_{b2}(2P)$ $\Upsilon(2S)$             0.6  
$\chi_{b0}(3P)$ $\Upsilon(2S)$               0  
$\chi_{b1}(3P)$ $\Upsilon(2S)$               0  
$\chi_{b2}(3P)$ $\Upsilon(2S)$             0.6

### $\psi(3S)$ and $\Upsilon(3S)$

In [207]:
for suff in [["", "E1"], ["E1", "ELW"]]:
    print(suff)
    res = reportRs("psi_3S", "scr", ("c-scr",), suffixes=suff)
    display(res)
    with open("export/width{}.psi_3S.c-scr.tex".format("-".join(suff)), "w") as f:
        f.write(res.to_latex(escape=False, bold_rows=True, column_format="l|l|r|r|r|r|r|r|r|r"))

['', 'E1']


PDG  Deng  $\Gamma$  $r^{(0)}$  $r^{(2)}$  \
In         Out                                                          
$\psi(3S)$ $\eta_{c}(1S)$   nan  6.76      9.62          1          0   
           $\chi_{c0}(1P)$  nan   6.7      17.6          1          0   
           $\chi_{c1}(1P)$  nan   6.7      32.6       0.54          0   
           $\chi_{c2}(1P)$  nan   2.5      35.5      0.111      0.577   
           $\eta_{c}(2S)$   nan  1.78      2.01          1          0   
           $\chi_{c0}(2P)$  nan    27      20.9          1          0   
           $\chi_{c1}(2P)$  nan    67      39.4      0.521          0   
           $\chi_{c2}(2P)$  nan    82      46.2       0.11      0.581   
           $\eta_{c}(3S)$   nan 0.088    0.0344          1          0   

                            E1-$\Gamma$  E1-$r^{(0)}$  E1-$r^{(2)}$  
In         Out                                                       
$\psi(3S)$ $\eta_{c}(1S)$          9.62             1             0  
           $\chi_{c0}(1P)$         23.9             1             0  
           $\chi_{c1}(1P)$         35.2           0.5             0  
           $\chi_{c2}(1P)$         34.1           0.1           0.6  
           $\eta_{c}(2S)$          2.01             1             0  
           $\chi_{c0}(2P)$           24             1             0  
           $\chi_{c1}(2P)$           41           0.5             0  
           $\chi_{c2}(2P)$         44.7           0.1           0.6  
           $\eta_{c}(3S)$        0.0344             1             0

['E1', 'ELW']


PDG  Deng  E1-$\Gamma$  E1-$r^{(0)}$  \
In         Out                                                     
$\psi(3S)$ $\eta_{c}(1S)$   nan  6.76         9.62             1   
           $\chi_{c0}(1P)$  nan   6.7         23.9             1   
           $\chi_{c1}(1P)$  nan   6.7         35.2           0.5   
           $\chi_{c2}(1P)$  nan   2.5         34.1           0.1   
           $\eta_{c}(2S)$   nan  1.78         2.01             1   
           $\chi_{c0}(2P)$  nan    27           24             1   
           $\chi_{c1}(2P)$  nan    67           41           0.5   
           $\chi_{c2}(2P)$  nan    82         44.7           0.1   
           $\eta_{c}(3S)$   nan 0.088       0.0344             1   

                            E1-$r^{(2)}$  ELW-$\Gamma$  ELW-$r^{(0)}$  \
In         Out                                                          
$\psi(3S)$ $\eta_{c}(1S)$              0           3.8              1   
           $\chi_{c0}(1P)$             0          7.73              1   
           $\chi_{c1}(1P)$             0         0.675            0.5   
           $\chi_{c2}(1P)$           0.6          1.92            0.1   
           $\eta_{c}(2S)$              0         0.714              1   
           $\chi_{c0}(2P)$             0          31.4              1   
           $\chi_{c1}(2P)$             0          45.7            0.5   
           $\chi_{c2}(2P)$           0.6          47.9            0.1   
           $\eta_{c}(3S)$              0        0.0345              1   

                            ELW-$r^{(2)}$  
In         Out                             
$\psi(3S)$ $\eta_{c}(1S)$               0  
           $\chi_{c0}(1P)$              0  
           $\chi_{c1}(1P)$              0  
           $\chi_{c2}(1P)$            0.6  
           $\eta_{c}(2S)$               0  
           $\chi_{c0}(2P)$              0  
           $\chi_{c1}(2P)$              0  
           $\chi_{c2}(2P)$            0.6  
           $\eta_{c}(3S)$               0

In [208]:
for suff in [["", "E1"], ["E1", "ELW"]]:
    print(suff)
    res = reportRs("psi_3S", "lin", ("c-lin",), suffixes=suff)
    display(res)
    with open("export/width{}.psi_3S.c-lin.tex".format("-".join(suff)), "w") as f:
        f.write(res.to_latex(escape=False, bold_rows=True, column_format="l|l|r|r|r|r|r|r|r|r"))

['', 'E1']


PDG  Deng  $\Gamma$  $r^{(0)}$  $r^{(2)}$  \
In         Out                                                          
$\psi(3S)$ $\eta_{c}(1S)$   nan  6.66      10.4          1          0   
           $\chi_{c0}(1P)$  nan   5.9      22.9          1          0   
           $\chi_{c1}(1P)$  nan     4      42.2      0.537          0   
           $\chi_{c2}(1P)$  nan  0.25      41.7      0.107      0.586   
           $\eta_{c}(2S)$   nan  1.65       2.8          1          0   
           $\chi_{c0}(2P)$  nan    19      26.8          1          0   
           $\chi_{c1}(2P)$  nan    55      54.8      0.524          0   
           $\chi_{c2}(2P)$  nan    67      64.9      0.111      0.578   
           $\eta_{c}(3S)$   nan 0.051    0.0527          1          0   

                            E1-$\Gamma$  E1-$r^{(0)}$  E1-$r^{(2)}$  
In         Out                                                       
$\psi(3S)$ $\eta_{c}(1S)$          10.4             1             0  
           $\chi_{c0}(1P)$         30.5             1             0  
           $\chi_{c1}(1P)$         45.3           0.5             0  
           $\chi_{c2}(1P)$         40.8           0.1           0.6  
           $\eta_{c}(2S)$           2.8             1             0  
           $\chi_{c0}(2P)$         31.2             1             0  
           $\chi_{c1}(2P)$         57.5           0.5             0  
           $\chi_{c2}(2P)$         62.5           0.1           0.6  
           $\eta_{c}(3S)$        0.0527             1             0

['E1', 'ELW']


PDG  Deng  E1-$\Gamma$  E1-$r^{(0)}$  \
In         Out                                                     
$\psi(3S)$ $\eta_{c}(1S)$   nan  6.66         10.4             1   
           $\chi_{c0}(1P)$  nan   5.9         30.5             1   
           $\chi_{c1}(1P)$  nan     4         45.3           0.5   
           $\chi_{c2}(1P)$  nan  0.25         40.8           0.1   
           $\eta_{c}(2S)$   nan  1.65          2.8             1   
           $\chi_{c0}(2P)$  nan    19         31.2             1   
           $\chi_{c1}(2P)$  nan    55         57.5           0.5   
           $\chi_{c2}(2P)$  nan    67         62.5           0.1   
           $\eta_{c}(3S)$   nan 0.051       0.0527             1   

                            E1-$r^{(2)}$  ELW-$\Gamma$  ELW-$r^{(0)}$  \
In         Out                                                          
$\psi(3S)$ $\eta_{c}(1S)$              0          4.22              1   
           $\chi_{c0}(1P)$             0           7.3              1   
           $\chi_{c1}(1P)$             0         0.105            0.5   
           $\chi_{c2}(1P)$           0.6           7.1            0.1   
           $\eta_{c}(2S)$              0         0.894              1   
           $\chi_{c0}(2P)$             0          41.9              1   
           $\chi_{c1}(2P)$             0          65.4            0.5   
           $\chi_{c2}(2P)$           0.6          67.7            0.1   
           $\eta_{c}(3S)$              0        0.0528              1   

                            ELW-$r^{(2)}$  
In         Out                             
$\psi(3S)$ $\eta_{c}(1S)$               0  
           $\chi_{c0}(1P)$              0  
           $\chi_{c1}(1P)$              0  
           $\chi_{c2}(1P)$            0.6  
           $\eta_{c}(2S)$               0  
           $\chi_{c0}(2P)$              0  
           $\chi_{c1}(2P)$              0  
           $\chi_{c2}(2P)$            0.6  
           $\eta_{c}(3S)$               0

In [209]:
for suff in [["", "E1"], ["E1", "ELW"]]:
    print(suff)
    res = reportRs("yps_3S", "scr", ("1P1D2D.b-scr", "nSnP.b-scr"), suffixes=suff)
    display(res)
    with open("export/width{}.yps_3S.b-scr.tex".format("-".join(suff)), "w") as f:
        f.write(res.to_latex(escape=False, bold_rows=True, column_format="l|l|r|r|r|r|r|r|r|r"))

['', 'E1']


PDG   Deng  $\Gamma$  $r^{(0)}$  $r^{(2)}$  \
In              Out                                                             
$\Upsilon(3S)$  $\chi_{b0}(1P)$ 0.0549  0.097     0.209          1          0   
                $\chi_{b1}(1P)$ 0.0183 0.0005    0.0841      0.499          0   
                $\chi_{b2}(1P)$  0.201   0.14  0.000345      0.463     0.0706   
                $\chi_{b0}(2P)$    1.2   1.21      1.23          1          0   
                $\chi_{b1}(2P)$   2.56   2.61      2.15      0.505          0   
                $\chi_{b2}(2P)$   2.66   3.16      2.44      0.102      0.595   
$\chi_{b0}(3P)$ $\Upsilon(3S)$     nan   7.95      9.64          1          0   
$\chi_{b1}(3P)$ $\Upsilon(3S)$     nan   10.3      11.9      0.491          0   
$\chi_{b2}(3P)$ $\Upsilon(3S)$     nan   10.8      12.1      0.094      0.612   

                                 E1-$\Gamma$  E1-$r^{(0)}$  E1-$r^{(2)}$  
In              Out                                                       
$\Upsilon(3S)$  $\chi_{b0}(1P)$        0.222             1             0  
                $\chi_{b1}(1P)$        0.084           0.5             0  
                $\chi_{b2}(1P)$     4.06e-05           0.1           0.6  
                $\chi_{b0}(2P)$         1.26             1             0  
                $\chi_{b1}(2P)$         2.18           0.5             0  
                $\chi_{b2}(2P)$         2.42           0.1           0.6  
$\chi_{b0}(3P)$ $\Upsilon(3S)$          9.34             1             0  
$\chi_{b1}(3P)$ $\Upsilon(3S)$          11.7           0.5             0  
$\chi_{b2}(3P)$ $\Upsilon(3S)$          12.3           0.1           0.6

['E1', 'ELW']


PDG   Deng  E1-$\Gamma$  E1-$r^{(0)}$  \
In              Out                                                        
$\Upsilon(3S)$  $\chi_{b0}(1P)$ 0.0549  0.097        0.222             1   
                $\chi_{b1}(1P)$ 0.0183 0.0005        0.084           0.5   
                $\chi_{b2}(1P)$  0.201   0.14     4.06e-05           0.1   
                $\chi_{b0}(2P)$    1.2   1.21         1.26             1   
                $\chi_{b1}(2P)$   2.56   2.61         2.18           0.5   
                $\chi_{b2}(2P)$   2.66   3.16         2.42           0.1   
$\chi_{b0}(3P)$ $\Upsilon(3S)$     nan   7.95         9.34             1   
$\chi_{b1}(3P)$ $\Upsilon(3S)$     nan   10.3         11.7           0.5   
$\chi_{b2}(3P)$ $\Upsilon(3S)$     nan   10.8         12.3           0.1   

                                 E1-$r^{(2)}$  ELW-$\Gamma$  ELW-$r^{(0)}$  \
In              Out                                                          
$\Upsilon(3S)$  $\chi_{b0}(1P)$             0        0.0394              1   
                $\chi_{b1}(1P)$             0        0.0357            0.5   
                $\chi_{b2}(1P)$           0.6         0.394            0.1   
                $\chi_{b0}(2P)$             0          1.31              1   
                $\chi_{b1}(2P)$             0          2.22            0.5   
                $\chi_{b2}(2P)$           0.6          2.46            0.1   
$\chi_{b0}(3P)$ $\Upsilon(3S)$              0            10              1   
$\chi_{b1}(3P)$ $\Upsilon(3S)$              0            13            0.5   
$\chi_{b2}(3P)$ $\Upsilon(3S)$            0.6            14            0.1   

                                 ELW-$r^{(2)}$  
In              Out                             
$\Upsilon(3S)$  $\chi_{b0}(1P)$              0  
                $\chi_{b1}(1P)$              0  
                $\chi_{b2}(1P)$            0.6  
                $\chi_{b0}(2P)$              0  
                $\chi_{b1}(2P)$              0  
                $\chi_{b2}(2P)$            0.6  
$\chi_{b0}(3P)$ $\Upsilon(3S)$               0  
$\chi_{b1}(3P)$ $\Upsilon(3S)$               0  
$\chi_{b2}(3P)$ $\Upsilon(3S)$             0.6

### $\psi_1(1D)$ which is $\psi(3770)$

In [210]:
for suff in [["", "E1"], ["E1", "ELW"]]:
    print(suff)
    res = reportRs("psi_1_1D", "scr", ("c-scr",), suffixes=suff)
    display(res)
    with open("export/width{}.psi_1_1D.c-scr.tex".format("-".join(suff)), "w") as f:
        f.write(res.to_latex(escape=False, bold_rows=True, column_format="l|l|r|r|r|r|r|r|r|r"))

['', 'E1']


PDG  Deng  $\Gamma$  $r^{(0)}$  $r^{(2)}$  \
In              Out                                                          
$\psi_{1}(1D)$  $\eta_{c}(1S)$   nan   nan     0.427          1          0   
                $\chi_{c0}(1P)$  190   261       288          1          0   
                $\chi_{c1}(1P)$ 67.5   135       152      0.466          0   
                $\chi_{c2}(1P)$  nan   8.1      9.94      0.123      0.737   
                $\eta_{c}(2S)$   nan   nan  0.000214          1          0   
$\chi_{c0}(2P)$ $\psi_{1}(1D)$   nan    12      6.87          1          0   
$\chi_{c1}(2P)$ $\psi_{1}(1D)$   nan   9.8      9.88      0.518          0   
$\chi_{c2}(2P)$ $\psi_{1}(1D)$   nan  0.46     0.466      0.232      0.582   
$\chi_{c0}(3P)$ $\psi_{1}(1D)$   nan  0.39      7.14          1          0   
$\chi_{c1}(3P)$ $\psi_{1}(1D)$   nan     2      9.42      0.535          0   
$\chi_{c2}(3P)$ $\psi_{1}(1D)$   nan  0.79      1.21     0.0869      0.034   

                                 E1-$\Gamma$  E1-$r^{(0)}$  E1-$r^{(2)}$  
In              Out                                                       
$\psi_{1}(1D)$  $\eta_{c}(1S)$         0.209             1             0  
                $\chi_{c0}(1P)$          253             1             0  
                $\chi_{c1}(1P)$          127           0.5             0  
                $\chi_{c2}(1P)$         7.02           0.1           0.6  
                $\eta_{c}(2S)$      0.000105             1             0  
$\chi_{c0}(2P)$ $\psi_{1}(1D)$          7.01             1             0  
$\chi_{c1}(2P)$ $\psi_{1}(1D)$          10.8           0.5             0  
$\chi_{c2}(2P)$ $\psi_{1}(1D)$         0.556           0.1           0.6  
$\chi_{c0}(3P)$ $\psi_{1}(1D)$          6.91             1             0  
$\chi_{c1}(3P)$ $\psi_{1}(1D)$          10.8           0.5             0  
$\chi_{c2}(3P)$ $\psi_{1}(1D)$         0.554           0.1           0.6

['E1', 'ELW']


PDG  Deng  E1-$\Gamma$  E1-$r^{(0)}$  \
In              Out                                                     
$\psi_{1}(1D)$  $\eta_{c}(1S)$   nan   nan        0.209             1   
                $\chi_{c0}(1P)$  190   261          253             1   
                $\chi_{c1}(1P)$ 67.5   135          127           0.5   
                $\chi_{c2}(1P)$  nan   8.1         7.02           0.1   
                $\eta_{c}(2S)$   nan   nan     0.000105             1   
$\chi_{c0}(2P)$ $\psi_{1}(1D)$   nan    12         7.01             1   
$\chi_{c1}(2P)$ $\psi_{1}(1D)$   nan   9.8         10.8           0.5   
$\chi_{c2}(2P)$ $\psi_{1}(1D)$   nan  0.46        0.556           0.1   
$\chi_{c0}(3P)$ $\psi_{1}(1D)$   nan  0.39         6.91             1   
$\chi_{c1}(3P)$ $\psi_{1}(1D)$   nan     2         10.8           0.5   
$\chi_{c2}(3P)$ $\psi_{1}(1D)$   nan  0.79        0.554           0.1   

                                 E1-$r^{(2)}$  ELW-$\Gamma$  ELW-$r^{(0)}$  \
In              Out                                                          
$\psi_{1}(1D)$  $\eta_{c}(1S)$              0             0            nan   
                $\chi_{c0}(1P)$             0           349              1   
                $\chi_{c1}(1P)$             0           153            0.5   
                $\chi_{c2}(1P)$           0.6          8.22            0.1   
                $\eta_{c}(2S)$              0             0            nan   
$\chi_{c0}(2P)$ $\psi_{1}(1D)$              0          7.12              1   
$\chi_{c1}(2P)$ $\psi_{1}(1D)$              0          11.8            0.5   
$\chi_{c2}(2P)$ $\psi_{1}(1D)$            0.6         0.638            0.1   
$\chi_{c0}(3P)$ $\psi_{1}(1D)$              0          8.48              1   
$\chi_{c1}(3P)$ $\psi_{1}(1D)$              0         0.355            0.5   
$\chi_{c2}(3P)$ $\psi_{1}(1D)$            0.6        0.0719            0.1   

                                 ELW-$r^{(2)}$  
In              Out                             
$\psi_{1}(1D)$  $\eta_{c}(1S)$             nan  
                $\chi_{c0}(1P)$              0  
                $\chi_{c1}(1P)$              0  
                $\chi_{c2}(1P)$            0.6  
                $\eta_{c}(2S)$             nan  
$\chi_{c0}(2P)$ $\psi_{1}(1D)$               0  
$\chi_{c1}(2P)$ $\psi_{1}(1D)$               0  
$\chi_{c2}(2P)$ $\psi_{1}(1D)$             0.6  
$\chi_{c0}(3P)$ $\psi_{1}(1D)$               0  
$\chi_{c1}(3P)$ $\psi_{1}(1D)$               0  
$\chi_{c2}(3P)$ $\psi_{1}(1D)$             0.6

In [211]:
for suff in [["", "E1"], ["E1", "ELW"]]:
    print(suff)
    res = reportRs("psi_1_1D", "lin", ("c-lin",), suffixes=suff)
    display(res)
    with open("export/width{}.psi_1_1D.c-lin.tex".format("-".join(suff)), "w") as f:
        f.write(res.to_latex(escape=False, bold_rows=True, column_format="l|l|r|r|r|r|r|r|r|r"))

['', 'E1']


PDG  Deng  $\Gamma$  $r^{(0)}$  $r^{(2)}$  \
In              Out                                                          
$\psi_{1}(1D)$  $\eta_{c}(1S)$   nan   nan     0.365          1          0   
                $\chi_{c0}(1P)$  190   272       288          1          0   
                $\chi_{c1}(1P)$ 67.5   138       146      0.468          0   
                $\chi_{c2}(1P)$  nan   7.1      8.87      0.117      0.728   
                $\eta_{c}(2S)$   nan   nan  0.000138          1          0   
$\chi_{c0}(2P)$ $\psi_{1}(1D)$   nan    20      16.4          1          0   
$\chi_{c1}(2P)$ $\psi_{1}(1D)$   nan   7.9      13.9      0.521          0   
$\chi_{c2}(2P)$ $\psi_{1}(1D)$   nan  0.36     0.626      0.281      0.576   
$\chi_{c0}(3P)$ $\psi_{1}(1D)$   nan  0.27      28.1          1          0   
$\chi_{c1}(3P)$ $\psi_{1}(1D)$   nan   3.2      18.3       0.54          0   
$\chi_{c2}(3P)$ $\psi_{1}(1D)$   nan   1.5      2.27     0.0927     0.0269   

                                 E1-$\Gamma$  E1-$r^{(0)}$  E1-$r^{(2)}$  
In              Out                                                       
$\psi_{1}(1D)$  $\eta_{c}(1S)$         0.179             1             0  
                $\chi_{c0}(1P)$          255             1             0  
                $\chi_{c1}(1P)$          123           0.5             0  
                $\chi_{c2}(1P)$         6.44           0.1           0.6  
                $\eta_{c}(2S)$      6.78e-05             1             0  
$\chi_{c0}(2P)$ $\psi_{1}(1D)$          16.9             1             0  
$\chi_{c1}(2P)$ $\psi_{1}(1D)$          15.5           0.5             0  
$\chi_{c2}(2P)$ $\psi_{1}(1D)$         0.753           0.1           0.6  
$\chi_{c0}(3P)$ $\psi_{1}(1D)$          28.3             1             0  
$\chi_{c1}(3P)$ $\psi_{1}(1D)$          21.3           0.5             0  
$\chi_{c2}(3P)$ $\psi_{1}(1D)$          1.02           0.1           0.6

['E1', 'ELW']


PDG  Deng  E1-$\Gamma$  E1-$r^{(0)}$  \
In              Out                                                     
$\psi_{1}(1D)$  $\eta_{c}(1S)$   nan   nan        0.179             1   
                $\chi_{c0}(1P)$  190   272          255             1   
                $\chi_{c1}(1P)$ 67.5   138          123           0.5   
                $\chi_{c2}(1P)$  nan   7.1         6.44           0.1   
                $\eta_{c}(2S)$   nan   nan     6.78e-05             1   
$\chi_{c0}(2P)$ $\psi_{1}(1D)$   nan    20         16.9             1   
$\chi_{c1}(2P)$ $\psi_{1}(1D)$   nan   7.9         15.5           0.5   
$\chi_{c2}(2P)$ $\psi_{1}(1D)$   nan  0.36        0.753           0.1   
$\chi_{c0}(3P)$ $\psi_{1}(1D)$   nan  0.27         28.3             1   
$\chi_{c1}(3P)$ $\psi_{1}(1D)$   nan   3.2         21.3           0.5   
$\chi_{c2}(3P)$ $\psi_{1}(1D)$   nan   1.5         1.02           0.1   

                                 E1-$r^{(2)}$  ELW-$\Gamma$  ELW-$r^{(0)}$  \
In              Out                                                          
$\psi_{1}(1D)$  $\eta_{c}(1S)$              0             0            nan   
                $\chi_{c0}(1P)$             0           339              1   
                $\chi_{c1}(1P)$             0           146            0.5   
                $\chi_{c2}(1P)$           0.6          7.39            0.1   
                $\eta_{c}(2S)$              0             0            nan   
$\chi_{c0}(2P)$ $\psi_{1}(1D)$              0          17.4              1   
$\chi_{c1}(2P)$ $\psi_{1}(1D)$              0          17.4            0.5   
$\chi_{c2}(2P)$ $\psi_{1}(1D)$            0.6         0.908            0.1   
$\chi_{c0}(3P)$ $\psi_{1}(1D)$              0          8.75              1   
$\chi_{c1}(3P)$ $\psi_{1}(1D)$              0         0.328            0.5   
$\chi_{c2}(3P)$ $\psi_{1}(1D)$            0.6        0.0873            0.1   

                                 ELW-$r^{(2)}$  
In              Out                             
$\psi_{1}(1D)$  $\eta_{c}(1S)$             nan  
                $\chi_{c0}(1P)$              0  
                $\chi_{c1}(1P)$              0  
                $\chi_{c2}(1P)$            0.6  
                $\eta_{c}(2S)$             nan  
$\chi_{c0}(2P)$ $\psi_{1}(1D)$               0  
$\chi_{c1}(2P)$ $\psi_{1}(1D)$               0  
$\chi_{c2}(2P)$ $\psi_{1}(1D)$             0.6  
$\chi_{c0}(3P)$ $\psi_{1}(1D)$               0  
$\chi_{c1}(3P)$ $\psi_{1}(1D)$               0  
$\chi_{c2}(3P)$ $\psi_{1}(1D)$             0.6

## Sumrules

In [212]:
def getMasses(prefix, cfgname):
    with open("../quarkEigen/output/{}.{}/config".format(prefix, cfgname), "r") as f:
        mass_cfg = json.load(f, object_pairs_hook=OrderedDict)
    with open("../quarkEigen/output/{}.{}/exclude".format(prefix, cfgname), "r") as f:
        mass_excl = f.read().split("\n")
    mass_data = OrderedDict()
    for particle, pdata in mass_cfg.items():
        if particle not in mass_excl:
            continue
        energ_data = pd.read_csv("../quarkEigen/output/{}.{}/data/{}/minE.dat".format(prefix, cfgname, particle))\
                     .iloc[-1, 1]
        energ = pdata["eq"]["env"]["mC"]*2 + energ_data
        mass_data.update({particle: energ})
    cfgcode, cfgtag = cfgnameSplit(cfgname)
    mass_data = pd.DataFrame.from_dict(mass_data, orient="index")
    mass_data.columns = ["E"]
    pdg = pd.read_csv("../data/{}-spectrum.csv".format(cfgcode.split("-")[0]), header=0, index_col=0)["PDG"]/10**3
    pdg = pdg.loc[pdg.index.intersection(mass_data.index).values]
    mass_data.loc[~pd.isna(pdg), "E"] = pdg.loc[~pd.isna(pdg)]
    mass_data.index = pd.Index(["${}$".format(code2name(k)) for k in mass_data.index.values], name="States")
    return mass_data

In [213]:
def getPprops(prefix, cfgname):
    with open("../quarkEigen/output/{}.{}/config".format(prefix, cfgname), "r") as f:
        pprops = json.load(f, object_pairs_hook=OrderedDict)
    props = pd.DataFrame(OrderedDict([(particle, values["eq"]) for particle, values in pprops.items()])).T
    props.drop(columns=["env", "E"], inplace=True)
    props.index = pd.Index([r"${}$".format(code2name(particle)) for particle in pprops.keys()], name="States")
    return props

In [214]:
def getSumrule(pname, prefix, cfgnames, suffix=""):
    cfgcode, cfgtag = cfgnameSplit(cfgnames[0])
    msuffix = "" if suffix == "" else suffix+"-"
    pretty_pname = "${}$".format(code2name(pname))
    
    widths = reportRs(pname, prefix, cfgnames, [suffix])
    masses = getMasses(prefix, cfgcode)
    pprops = getPprops(prefix, cfgcode)
    
    sumrule = []

    abovethr = [p[0] for p in widths.query("Out == r'{}'".format(pretty_pname)).index.values]
    sumrule.append(
        8*sp.pi**2*masses.loc[abovethr, "E"]**3\
        /(masses.loc[abovethr, "E"]**2 - masses.loc[pretty_pname, "E"]**2)**3\
        *pprops.loc[abovethr, "xJ"]
    )
    sumrule[0].index = pd.Index([(v, pretty_pname) for v in sumrule[0].index.values])
    
    belowthr = [p[1] for p in widths.query("In == r'{}'".format(pretty_pname)).index.values]
    sumrule.append(
        8*sp.pi**2*masses.loc[pretty_pname, "E"]**3\
        /(masses.loc[pretty_pname, "E"]**2 - masses.loc[belowthr, "E"]**2)**3\
        *pprops.loc[pretty_pname, "xJ"]
    )
    sumrule[1].index = pd.Index([(pretty_pname, v) for v in sumrule[1].index.values])
    sr = pd.concat(sumrule)*(0.19732697)**2*10**4*10**(-6)
    sr = sr.reindex(widths.index.values)
    
    widths["$SR-PDG$"] = widths["PDG"]
    if sp.any(pd.isna(widths["PDG"])):
        widths.loc[pd.isna(widths["PDG"]), "$SR-PDG$"] = widths.loc[pd.isna(widths["PDG"]).index.values, msuffix+"$\Gamma$"]
    widths.loc[:, "$SR-PDG$"] *= sr*(widths.loc[:, msuffix+"$r^{(2)}$"] - widths.loc[:, msuffix+"$r^{(0)}$"])
    
    widths["$SR-Deng$"] = widths["Deng"]
    if sp.any(pd.isna(widths["Deng"])):
        widths.loc[pd.isna(widths["Deng"]), "$SR-Deng$"] = widths.loc[pd.isna(widths["Deng"]).index.values, msuffix+"$\Gamma$"]
    widths.loc[:, "$SR-Deng$"] *= sr*(widths.loc[:, msuffix+"$r^{(2)}$"] - widths.loc[:, msuffix+"$r^{(0)}$"])
    
    widths["$SR-\Gamma$"] = widths[msuffix+"$\Gamma$"]*sr*(widths.loc[:, msuffix+"$r^{(2)}$"] - widths.loc[:, msuffix+"$r^{(0)}$"])
    
    widths.drop(columns=[msuffix+'$r^{(2)}$', msuffix+'$r^{(0)}$'], inplace=True)
    widths = widths[[widths.columns[i] for i in [0, 3, 1, 4, 2, 5]]]
    widths = dfsort(widths, lambda a,b: cmpMultyStatesByCode(prefix, cfgcode
                                                         ,[name2code(i[1:-1]) for i in a]
                                                         ,[name2code(i[1:-1]) for i in b]))
    
    return widths, pd.DataFrame(widths[["$SR-PDG$", "$SR-Deng$", "$SR-\Gamma$"]].sum().rename("$SR$"))

In [229]:
def reportSumrules(pname, prefix, cfgnames, suffixes=("", "E1", "ELW"), sigma=None):
    cfgcode, cfgtag = cfgnameSplit(cfgnames[0])
    for approx in suffixes:
        display("Approximation: "+(approx if len(approx) > 0 else "tot"))
        report, sr = getSumrule(pname, prefix, cfgnames, approx)
        display(sr)
        display(report)
        
        tab = Tabular("|l|l|c|c|c|c|c|c|")
        
        # Header
        tab.add_hline()
        tab.add_row(["", ""] + [NoEscape(i) for i in report.columns])
        tab.add_row([NoEscape(i) for i in report.index.names] + [""]*6)
        tab.add_hline()
        
        curN = "1"
        tot = report.iloc[0, 1::2]*0
        getN = re.compile("\((\d+)[A-Z]\)\$$")
        for entry in report.index.values:
            # Manage subtotals
            label = entry[0] if entry[0] != "${}$".format(code2name(pname)) else entry[1]
            newN = getN.search(label).groups()[0]
            if newN != curN:
                tab.add_hline()
                tab.add_hline()
                try:
                    tab.add_row([MultiColumn(2, align="|c|", data="Subtotal")] +\
                            [MultiColumn(2, align="|r|", data=NoEscape("${:g} ({:g}\sigma)$".format(num, num/sigma))) for num in tot])
                except:
                    tab.add_row([MultiColumn(2, align="|c|", data="Subtotal")] +\
                            [MultiColumn(2, align="|r|", data="{:g}".format(num)) for num in tot])                
                tab.add_hline()
                tab.add_hline()
                tot = report.loc[entry].values[1::2]
                curN = newN
            else:
                tot += report.loc[entry].values[1::2]
            
            # Regular entry
            tab.add_row([NoEscape(i) for i in list(entry) + ["{:g}".format(num) for num in report.loc[entry]]])
            
        # Final subtotal
        tab.add_hline()
        tab.add_hline()
        try:
            tab.add_row([MultiColumn(2, align="|c|", data="Subtotal")] +\
                    [MultiColumn(2, align="|r|", data=NoEscape("${:g}~({:g}\sigma)$".format(num, num/sigma))) for num in tot])
        except:
            tab.add_row([MultiColumn(2, align="|c|", data="Subtotal")] +\
                    [MultiColumn(2, align="|r|", data="{:g}".format(num)) for num in tot])                
        tab.add_hline()
        tab.add_hline()
        
        # Total
        try:
            tab.add_row([MultiColumn(2, align="|c|", data="Total")] +\
                    [MultiColumn(2, align="|r|", data=NoEscape("${:g}~({:g}\sigma)$".format(num, num/sigma))) for num in sr.iloc[:, 0]])
        except:
            tab.add_row([MultiColumn(2, align="|c|", data="Total")] +\
                    [MultiColumn(2, align="|r|", data="{:g}".format(num)) for num in sr.iloc[:, 0]])                
        tab.add_hline()
        
        # with open("export/sr{}.{}.{}.tex".format(approx, pname, cfgcode), "w") as f:
        #     f.write(sr.to_latex(escape=False, bold_rows=True, column_format="l|c"))
        # with open("export/sr{}-report.{}.{}.tex".format(approx, pname, cfgcode), "w") as f:
        #     f.write(report.to_latex(escape=False, bold_rows=True, column_format="l|l|c|c|c|c|c|c"))
        tab.generate_tex("export/sr{}-report.{}.{}".format(approx, pname, cfgcode))

## Bottomonium sumrules

### $\Upsilon(1S)$

In [216]:
reportSumrules("yps_1S", "scr", ("1P1D2D.b-scr", "nSnP.b-scr"), sigma=0.012)

'Approximation: tot'

,$SR$
$SR-PDG$,-0.0885
$SR-Deng$,-0.0766
$SR-\Gamma$,-0.0885


,,PDG,$SR-PDG$,Deng,$SR-Deng$,$\Gamma$,$SR-\Gamma$
In,Out,,,,,,
$\chi_{b0}(1P)$,$\Upsilon(1S)$,nan,-1.82,27.5,-1.77,28.3,-1.82
$\chi_{b1}(1P)$,$\Upsilon(1S)$,nan,-2.43,31.9,-2.31,33.6,-2.43
$\chi_{b2}(1P)$,$\Upsilon(1S)$,nan,3.97,31.8,3.85,32.8,3.97
$\chi_{b0}(2P)$,$\Upsilon(1S)$,nan,-0.0744,5.54,-0.0517,7.97,-0.0744
$\chi_{b1}(2P)$,$\Upsilon(1S)$,nan,-0.18,10.8,-0.13,15,-0.18
$\chi_{b2}(2P)$,$\Upsilon(1S)$,nan,0.39,12.5,0.289,16.8,0.39
$\chi_{b0}(3P)$,$\Upsilon(1S)$,nan,-0.0151,1.87,-0.00762,3.71,-0.0151
$\chi_{b1}(3P)$,$\Upsilon(1S)$,nan,-0.0511,6.41,-0.034,9.62,-0.0511
$\chi_{b2}(3P)$,$\Upsilon(1S)$,nan,0.125,8.17,0.0874,11.7,0.125


'Approximation: E1'

,$SR$
$SR-PDG$,-0.102
$SR-Deng$,-0.549
$SR-\Gamma$,-0.102


,,PDG,$SR-PDG$,Deng,$SR-Deng$,E1-$\Gamma$,$SR-\Gamma$
In,Out,,,,,,
$\chi_{b0}(1P)$,$\Upsilon(1S)$,nan,-1.68,27.5,-1.77,26.1,-1.68
$\chi_{b1}(1P)$,$\Upsilon(1S)$,nan,-2.43,31.9,-2.42,32.1,-2.43
$\chi_{b2}(1P)$,$\Upsilon(1S)$,nan,3.83,31.8,3.55,34.4,3.83
$\chi_{b0}(2P)$,$\Upsilon(1S)$,nan,-0.066,5.54,-0.0517,7.07,-0.066
$\chi_{b1}(2P)$,$\Upsilon(1S)$,nan,-0.18,10.8,-0.14,14,-0.18
$\chi_{b2}(2P)$,$\Upsilon(1S)$,nan,0.37,12.5,0.256,18.1,0.37
$\chi_{b0}(3P)$,$\Upsilon(1S)$,nan,-0.0132,1.87,-0.00762,3.24,-0.0132
$\chi_{b1}(3P)$,$\Upsilon(1S)$,nan,-0.0511,6.41,-0.037,8.84,-0.0511
$\chi_{b2}(3P)$,$\Upsilon(1S)$,nan,0.117,8.17,0.0753,12.7,0.117


'Approximation: ELW'

,$SR$
$SR-PDG$,-0.0587
$SR-Deng$,-0.549
$SR-\Gamma$,-0.0587


,,PDG,$SR-PDG$,Deng,$SR-Deng$,ELW-$\Gamma$,$SR-\Gamma$
In,Out,,,,,,
$\chi_{b0}(1P)$,$\Upsilon(1S)$,nan,-1.79,27.5,-1.77,27.9,-1.79
$\chi_{b1}(1P)$,$\Upsilon(1S)$,nan,-2.65,31.9,-2.42,34.9,-2.65
$\chi_{b2}(1P)$,$\Upsilon(1S)$,nan,4.22,31.8,3.55,37.8,4.22
$\chi_{b0}(2P)$,$\Upsilon(1S)$,nan,-0.0354,5.54,-0.0517,3.79,-0.0354
$\chi_{b1}(2P)$,$\Upsilon(1S)$,nan,-0.127,10.8,-0.14,9.79,-0.127
$\chi_{b2}(2P)$,$\Upsilon(1S)$,nan,0.283,12.5,0.256,13.8,0.283
$\chi_{b0}(3P)$,$\Upsilon(1S)$,nan,-0.00412,1.87,-0.00762,1.01,-0.00412
$\chi_{b1}(3P)$,$\Upsilon(1S)$,nan,-0.0281,6.41,-0.037,4.87,-0.0281
$\chi_{b2}(3P)$,$\Upsilon(1S)$,nan,0.0746,8.17,0.0753,8.1,0.0746


### $\Upsilon(2S)$

In [217]:
reportSumrules("yps_2S", "scr", ("1P1D2D.b-scr", "nSnP.b-scr"), sigma=1.01)

'Approximation: tot'

,$SR$
$SR-PDG$,-1.37
$SR-Deng$,-0.134
$SR-\Gamma$,-1.15


PDG $SR-PDG$  Deng $SR-Deng$  $\Gamma$  \
In              Out                                                       
$\Upsilon(2S)$  $\chi_{b0}(1P)$ 1.22    -3.26  1.09     -2.92     0.861   
                $\chi_{b1}(1P)$ 2.21    -5.97  2.17     -5.87       1.4   
                $\chi_{b2}(1P)$ 2.29     9.63  2.62        11      1.59   
$\chi_{b0}(2P)$ $\Upsilon(2S)$   nan    -5.57  14.4     -6.16        13   
$\chi_{b1}(2P)$ $\Upsilon(2S)$   nan    -7.19  15.3     -7.13      15.4   
$\chi_{b2}(2P)$ $\Upsilon(2S)$   nan     10.8  15.3      10.7      15.4   
$\chi_{b0}(3P)$ $\Upsilon(2S)$   nan   -0.182  2.55    -0.102      4.54   
$\chi_{b1}(3P)$ $\Upsilon(2S)$   nan   -0.415  5.63    -0.285      8.18   
$\chi_{b2}(3P)$ $\Upsilon(2S)$   nan      0.8  6.72     0.583      9.23   

                                $SR-\Gamma$  
In              Out                          
$\Upsilon(2S)$  $\chi_{b0}(1P)$       -2.31  
                $\chi_{b1}(1P)$       -3.78  
                $\chi_{b2}(1P)$        6.71  
$\chi_{b0}(2P)$ $\Upsilon(2S)$        -5.57  
$\chi_{b1}(2P)$ $\Upsilon(2S)$        -7.19  
$\chi_{b2}(2P)$ $\Upsilon(2S)$         10.8  
$\chi_{b0}(3P)$ $\Upsilon(2S)$       -0.182  
$\chi_{b1}(3P)$ $\Upsilon(2S)$       -0.415  
$\chi_{b2}(3P)$ $\Upsilon(2S)$          0.8

'Approximation: E1'

,$SR$
$SR-PDG$,-1.13
$SR-Deng$,-0.593
$SR-\Gamma$,-1.17


PDG $SR-PDG$  Deng $SR-Deng$  E1-$\Gamma$  \
In              Out                                                          
$\Upsilon(2S)$  $\chi_{b0}(1P)$ 1.22    -3.26  1.09     -2.92        0.889   
                $\chi_{b1}(1P)$ 2.21     -5.9  2.17     -5.81         1.41   
                $\chi_{b2}(1P)$ 2.29      9.8  2.62      11.2         1.58   
$\chi_{b0}(2P)$ $\Upsilon(2S)$   nan    -5.34  14.4     -6.16         12.5   
$\chi_{b1}(2P)$ $\Upsilon(2S)$   nan    -7.19  15.3     -7.31         15.1   
$\chi_{b2}(2P)$ $\Upsilon(2S)$   nan     10.6  15.3      10.2         15.8   
$\chi_{b0}(3P)$ $\Upsilon(2S)$   nan   -0.169  2.55    -0.102         4.23   
$\chi_{b1}(3P)$ $\Upsilon(2S)$   nan   -0.415  5.63    -0.298         7.84   
$\chi_{b2}(3P)$ $\Upsilon(2S)$   nan    0.773  6.72     0.538         9.65   

                                $SR-\Gamma$  
In              Out                          
$\Upsilon(2S)$  $\chi_{b0}(1P)$       -2.38  
                $\chi_{b1}(1P)$       -3.78  
                $\chi_{b2}(1P)$        6.76  
$\chi_{b0}(2P)$ $\Upsilon(2S)$        -5.34  
$\chi_{b1}(2P)$ $\Upsilon(2S)$        -7.19  
$\chi_{b2}(2P)$ $\Upsilon(2S)$         10.6  
$\chi_{b0}(3P)$ $\Upsilon(2S)$       -0.169  
$\chi_{b1}(3P)$ $\Upsilon(2S)$       -0.415  
$\chi_{b2}(3P)$ $\Upsilon(2S)$        0.773

'Approximation: ELW'

,$SR$
$SR-PDG$,-0.944
$SR-Deng$,-0.593
$SR-\Gamma$,-1.03


PDG $SR-PDG$  Deng $SR-Deng$  ELW-$\Gamma$  \
In              Out                                                           
$\Upsilon(2S)$  $\chi_{b0}(1P)$ 1.22    -3.26  1.09     -2.92         0.909   
                $\chi_{b1}(1P)$ 2.21     -5.9  2.17     -5.81          1.43   
                $\chi_{b2}(1P)$ 2.29      9.8  2.62      11.2          1.59   
$\chi_{b0}(2P)$ $\Upsilon(2S)$   nan     -5.7  14.4     -6.16          13.3   
$\chi_{b1}(2P)$ $\Upsilon(2S)$   nan     -7.9  15.3     -7.31          16.5   
$\chi_{b2}(2P)$ $\Upsilon(2S)$   nan     11.8  15.3      10.2          17.7   
$\chi_{b0}(3P)$ $\Upsilon(2S)$   nan  -0.0675  2.55    -0.102          1.69   
$\chi_{b1}(3P)$ $\Upsilon(2S)$   nan   -0.239  5.63    -0.298          4.52   
$\chi_{b2}(3P)$ $\Upsilon(2S)$   nan    0.502  6.72     0.538          6.26   

                                $SR-\Gamma$  
In              Out                          
$\Upsilon(2S)$  $\chi_{b0}(1P)$       -2.44  
                $\chi_{b1}(1P)$       -3.83  
                $\chi_{b2}(1P)$        6.83  
$\chi_{b0}(2P)$ $\Upsilon(2S)$         -5.7  
$\chi_{b1}(2P)$ $\Upsilon(2S)$         -7.9  
$\chi_{b2}(2P)$ $\Upsilon(2S)$         11.8  
$\chi_{b0}(3P)$ $\Upsilon(2S)$      -0.0675  
$\chi_{b1}(3P)$ $\Upsilon(2S)$       -0.239  
$\chi_{b2}(3P)$ $\Upsilon(2S)$        0.502

### $\Upsilon(3S)$

In [218]:
reportSumrules("yps_3S", "scr", ("1P1D2D.b-scr", "nSnP.b-scr"))

'Approximation: tot'

,$SR$
$SR-PDG$,-7.81
$SR-Deng$,-1.18
$SR-\Gamma$,-7.6


PDG $SR-PDG$   Deng $SR-Deng$  $\Gamma$  \
In              Out                                                          
$\Upsilon(3S)$  $\chi_{b0}(1P)$ 0.0549 -0.00557  0.097  -0.00986     0.209   
                $\chi_{b1}(1P)$ 0.0183 -0.00114 0.0005 -3.12e-05    0.0841   
                $\chi_{b2}(1P)$  0.201  -0.0112   0.14  -0.00777  0.000345   
                $\chi_{b0}(2P)$    1.2    -7.75   1.21     -7.82      1.23   
                $\chi_{b1}(2P)$   2.56    -15.1   2.61     -15.4      2.15   
                $\chi_{b2}(2P)$   2.66     24.1   3.16      28.6      2.44   
$\chi_{b0}(3P)$ $\Upsilon(3S)$     nan    -14.9   7.95     -12.3      9.64   
$\chi_{b1}(3P)$ $\Upsilon(3S)$     nan    -17.8   10.3     -15.4      11.9   
$\chi_{b2}(3P)$ $\Upsilon(3S)$     nan     23.7   10.8      21.2      12.1   

                                $SR-\Gamma$  
In              Out                          
$\Upsilon(3S)$  $\chi_{b0}(1P)$     -0.0212  
                $\chi_{b1}(1P)$    -0.00525  
                $\chi_{b2}(1P)$   -1.92e-05  
                $\chi_{b0}(2P)$       -7.94  
                $\chi_{b1}(2P)$       -12.7  
                $\chi_{b2}(2P)$        22.1  
$\chi_{b0}(3P)$ $\Upsilon(3S)$        -14.9  
$\chi_{b1}(3P)$ $\Upsilon(3S)$        -17.8  
$\chi_{b2}(3P)$ $\Upsilon(3S)$         23.7

'Approximation: E1'

,$SR$
$SR-PDG$,-7.18
$SR-Deng$,-1.6
$SR-\Gamma$,-7.57


PDG $SR-PDG$   Deng $SR-Deng$  E1-$\Gamma$  \
In              Out                                                             
$\Upsilon(3S)$  $\chi_{b0}(1P)$ 0.0549 -0.00557  0.097  -0.00986        0.222   
                $\chi_{b1}(1P)$ 0.0183 -0.00114 0.0005 -3.13e-05        0.084   
                $\chi_{b2}(1P)$  0.201   0.0142   0.14    0.0099     4.06e-05   
                $\chi_{b0}(2P)$    1.2    -7.75   1.21     -7.82         1.26   
                $\chi_{b1}(2P)$   2.56      -15   2.61     -15.3         2.18   
                $\chi_{b2}(2P)$   2.66     24.4   3.16        29         2.42   
$\chi_{b0}(3P)$ $\Upsilon(3S)$     nan    -14.4   7.95     -12.3         9.34   
$\chi_{b1}(3P)$ $\Upsilon(3S)$     nan    -17.8   10.3     -15.7         11.7   
$\chi_{b2}(3P)$ $\Upsilon(3S)$     nan     23.3   10.8      20.5         12.3   

                                $SR-\Gamma$  
In              Out                          
$\Upsilon(3S)$  $\chi_{b0}(1P)$     -0.0226  
                $\chi_{b1}(1P)$    -0.00525  
                $\chi_{b2}(1P)$    2.87e-06  
                $\chi_{b0}(2P)$       -8.15  
                $\chi_{b1}(2P)$       -12.7  
                $\chi_{b2}(2P)$        22.2  
$\chi_{b0}(3P)$ $\Upsilon(3S)$        -14.4  
$\chi_{b1}(3P)$ $\Upsilon(3S)$        -17.8  
$\chi_{b2}(3P)$ $\Upsilon(3S)$         23.3

'Approximation: ELW'

,$SR$
$SR-PDG$,-7.03
$SR-Deng$,-1.6
$SR-\Gamma$,-7.61


PDG $SR-PDG$   Deng $SR-Deng$  \
In              Out                                                
$\Upsilon(3S)$  $\chi_{b0}(1P)$ 0.0549 -0.00557  0.097  -0.00986   
                $\chi_{b1}(1P)$ 0.0183 -0.00114 0.0005 -3.13e-05   
                $\chi_{b2}(1P)$  0.201   0.0142   0.14    0.0099   
                $\chi_{b0}(2P)$    1.2    -7.75   1.21     -7.82   
                $\chi_{b1}(2P)$   2.56      -15   2.61     -15.3   
                $\chi_{b2}(2P)$   2.66     24.4   3.16        29   
$\chi_{b0}(3P)$ $\Upsilon(3S)$     nan    -15.5   7.95     -12.3   
$\chi_{b1}(3P)$ $\Upsilon(3S)$     nan    -19.7   10.3     -15.7   
$\chi_{b2}(3P)$ $\Upsilon(3S)$     nan     26.5   10.8      20.5   

                                 ELW-$\Gamma$ $SR-\Gamma$  
In              Out                                        
$\Upsilon(3S)$  $\chi_{b0}(1P)$        0.0394      -0.004  
                $\chi_{b1}(1P)$        0.0357    -0.00223  
                $\chi_{b2}(1P)$         0.394      0.0279  
                $\chi_{b0}(2P)$          1.31       -8.46  
                $\chi_{b1}(2P)$          2.22         -13  
                $\chi_{b2}(2P)$          2.46        22.6  
$\chi_{b0}(3P)$ $\Upsilon(3S)$             10       -15.5  
$\chi_{b1}(3P)$ $\Upsilon(3S)$             13       -19.7  
$\chi_{b2}(3P)$ $\Upsilon(3S)$             14        26.5

## Charmonium sumrules

### $\psi(1S)$

In [219]:
reportSumrules("psi_1S", "scr", ("c-scr",), sigma=6.7)

'Approximation: tot'

,$SR$
$SR-PDG$,5.64
$SR-Deng$,-25
$SR-\Gamma$,-31.3


,,PDG,$SR-PDG$,Deng,$SR-Deng$,$\Gamma$,$SR-\Gamma$
In,Out,,,,,,
$\psi(1S)$,$\eta_{c}(1S)$,1.58,-13.3,2.44,-20.6,2.78,-23.5
$\chi_{c0}(1P)$,$\psi(1S)$,133,-18.4,179,-24.7,204,-28.2
$\chi_{c1}(1P)$,$\psi(1S)$,285,-24.1,319,-27,376,-31.8
$\chi_{c2}(1P)$,$\psi(1S)$,371,57.7,292,45.4,311,48.4
$\eta_{c}(2S)$,$\psi(1S)$,nan,-0.0887,2.29,-0.0697,2.91,-0.0887
$\chi_{c0}(2P)$,$\psi(1S)$,nan,-0.483,2.3,-0.0223,49.9,-0.483
$\chi_{c1}(2P)$,$\psi(1S)$,nan,-3.04,88,-1.09,244,-3.04
$\chi_{c2}(2P)$,$\psi(1S)$,nan,7.37,93,2.97,231,7.37


'Approximation: E1'

,$SR$
$SR-PDG$,-11.5
$SR-Deng$,-40.1
$SR-\Gamma$,-31.5


,,PDG,$SR-PDG$,Deng,$SR-Deng$,E1-$\Gamma$,$SR-\Gamma$
In,Out,,,,,,
$\psi(1S)$,$\eta_{c}(1S)$,1.58,-13.3,2.44,-20.6,2.78,-23.5
$\chi_{c0}(1P)$,$\psi(1S)$,133,-18.4,179,-24.7,167,-23
$\chi_{c1}(1P)$,$\psi(1S)$,285,-27.8,319,-31.1,326,-31.8
$\chi_{c2}(1P)$,$\psi(1S)$,371,45,292,35.4,361,43.8
$\eta_{c}(2S)$,$\psi(1S)$,nan,-0.0887,2.29,-0.0697,2.91,-0.0887
$\chi_{c0}(2P)$,$\psi(1S)$,nan,-0.435,2.3,-0.0223,45,-0.435
$\chi_{c1}(2P)$,$\psi(1S)$,nan,-3.04,88,-1.3,206,-3.04
$\chi_{c2}(2P)$,$\psi(1S)$,nan,6.54,93,2.18,279,6.54


'Approximation: ELW'

,$SR$
$SR-PDG$,-12.5
$SR-Deng$,-40.1
$SR-\Gamma$,-29.1


,,PDG,$SR-PDG$,Deng,$SR-Deng$,ELW-$\Gamma$,$SR-\Gamma$
In,Out,,,,,,
$\psi(1S)$,$\eta_{c}(1S)$,1.58,-13.3,2.44,-20.6,2.8,-23.6
$\chi_{c0}(1P)$,$\psi(1S)$,133,-18.4,179,-24.7,187,-25.8
$\chi_{c1}(1P)$,$\psi(1S)$,285,-27.8,319,-31.1,411,-40.1
$\chi_{c2}(1P)$,$\psi(1S)$,371,45,292,35.4,481,58.4
$\eta_{c}(2S)$,$\psi(1S)$,nan,-0.28,2.29,-0.0697,9.19,-0.28
$\chi_{c0}(2P)$,$\psi(1S)$,nan,-0.0305,2.3,-0.0223,3.15,-0.0305
$\chi_{c1}(2P)$,$\psi(1S)$,nan,-0.884,88,-1.3,60.1,-0.884
$\chi_{c2}(2P)$,$\psi(1S)$,nan,3.2,93,2.18,136,3.2


In [220]:
reportSumrules("psi_1S", "lin", ("c-lin",), sigma=16.6)

'Approximation: tot'

,$SR$
$SR-PDG$,5.57
$SR-Deng$,-24.5
$SR-\Gamma$,-27.3


,,PDG,$SR-PDG$,Deng,$SR-Deng$,$\Gamma$,$SR-\Gamma$
In,Out,,,,,,
$\psi(1S)$,$\eta_{c}(1S)$,1.58,-13.3,2.39,-20.2,2.57,-21.7
$\chi_{c0}(1P)$,$\psi(1S)$,133,-18.4,172,-23.7,195,-26.9
$\chi_{c1}(1P)$,$\psi(1S)$,285,-24.3,306,-26.1,350,-29.9
$\chi_{c2}(1P)$,$\psi(1S)$,371,57,284,43.7,303,46.6
$\eta_{c}(2S)$,$\psi(1S)$,nan,-0.0923,2.64,-0.0804,3.03,-0.0923
$\chi_{c0}(2P)$,$\psi(1S)$,nan,-0.584,6.1,-0.059,60.4,-0.584
$\chi_{c1}(2P)$,$\psi(1S)$,nan,-3,81,-0.94,259,-3
$\chi_{c2}(2P)$,$\psi(1S)$,nan,8.31,93,2.94,263,8.31


'Approximation: E1'

,$SR$
$SR-PDG$,-10.7
$SR-Deng$,-38.3
$SR-\Gamma$,-27.6


,,PDG,$SR-PDG$,Deng,$SR-Deng$,E1-$\Gamma$,$SR-\Gamma$
In,Out,,,,,,
$\psi(1S)$,$\eta_{c}(1S)$,1.58,-13.3,2.39,-20.2,2.57,-21.7
$\chi_{c0}(1P)$,$\psi(1S)$,133,-18.4,172,-23.7,161,-22.2
$\chi_{c1}(1P)$,$\psi(1S)$,285,-27.8,306,-29.8,306,-29.9
$\chi_{c2}(1P)$,$\psi(1S)$,371,45,284,34.5,349,42.3
$\eta_{c}(2S)$,$\psi(1S)$,nan,-0.0923,2.64,-0.0804,3.03,-0.0923
$\chi_{c0}(2P)$,$\psi(1S)$,nan,-0.518,6.1,-0.059,53.5,-0.518
$\chi_{c1}(2P)$,$\psi(1S)$,nan,-3,81,-1.1,220,-3
$\chi_{c2}(2P)$,$\psi(1S)$,nan,7.4,93,2.18,315,7.4


'Approximation: ELW'

,$SR$
$SR-PDG$,-12.1
$SR-Deng$,-38.3
$SR-\Gamma$,-25.7


,,PDG,$SR-PDG$,Deng,$SR-Deng$,ELW-$\Gamma$,$SR-\Gamma$
In,Out,,,,,,
$\psi(1S)$,$\eta_{c}(1S)$,1.58,-13.3,2.39,-20.2,2.58,-21.8
$\chi_{c0}(1P)$,$\psi(1S)$,133,-18.4,172,-23.7,180,-24.8
$\chi_{c1}(1P)$,$\psi(1S)$,285,-27.8,306,-29.8,380,-37
$\chi_{c2}(1P)$,$\psi(1S)$,371,45,284,34.5,458,55.6
$\eta_{c}(2S)$,$\psi(1S)$,nan,-0.264,2.64,-0.0804,8.66,-0.264
$\chi_{c0}(2P)$,$\psi(1S)$,nan,-0.0166,6.1,-0.059,1.71,-0.0166
$\chi_{c1}(2P)$,$\psi(1S)$,nan,-0.79,81,-1.1,58,-0.79
$\chi_{c2}(2P)$,$\psi(1S)$,nan,3.43,93,2.18,146,3.43


### $\psi(2S)$

In [221]:
reportSumrules("psi_2S", "scr", ("c-scr",))

'Approximation: tot'

,$SR$
$SR-PDG$,4.66
$SR-Deng$,53.8
$SR-\Gamma$,49.4


PDG $SR-PDG$  Deng $SR-Deng$  $\Gamma$  \
In              Out                                                        
$\psi(2S)$      $\eta_{c}(1S)$   1.01  -0.0453   7.8    -0.351      10.3   
                $\chi_{c0}(1P)$  29.6    -19.2    22     -14.3      20.2   
                $\chi_{c1}(1P)$  28.3    -34.5    45     -54.9      34.9   
                $\chi_{c2}(1P)$    27     68.8    46       117      42.8   
                $\eta_{c}(2S)$  0.207    -23.5  0.19     -21.5     0.148   
$\chi_{c0}(2P)$ $\psi(2S)$        nan    -38.9    99     -33.3       116   
$\chi_{c1}(2P)$ $\psi(2S)$        nan    -92.6   155     -75.7       189   
$\chi_{c2}(2P)$ $\psi(2S)$        nan      141   150       133       159   
$\chi_{c0}(3P)$ $\psi(2S)$        nan    -3.38   9.1    -0.425      72.4   
$\chi_{c1}(3P)$ $\psi(2S)$        nan    -8.53    74     -3.53       179   
$\chi_{c2}(3P)$ $\psi(2S)$        nan     15.7    76      7.66       156   

                                $SR-\Gamma$  
In              Out                          
$\psi(2S)$      $\eta_{c}(1S)$       -0.462  
                $\chi_{c0}(1P)$       -13.1  
                $\chi_{c1}(1P)$       -42.6  
                $\chi_{c2}(1P)$         109  
                $\eta_{c}(2S)$        -16.8  
$\chi_{c0}(2P)$ $\psi(2S)$            -38.9  
$\chi_{c1}(2P)$ $\psi(2S)$            -92.6  
$\chi_{c2}(2P)$ $\psi(2S)$              141  
$\chi_{c0}(3P)$ $\psi(2S)$            -3.38  
$\chi_{c1}(3P)$ $\psi(2S)$            -8.53  
$\chi_{c2}(3P)$ $\psi(2S)$             15.7

'Approximation: E1'

,$SR$
$SR-PDG$,6.45
$SR-Deng$,38.1
$SR-\Gamma$,44.9


PDG $SR-PDG$  Deng $SR-Deng$  E1-$\Gamma$  \
In              Out                                                           
$\psi(2S)$      $\eta_{c}(1S)$   1.01  -0.0453   7.8    -0.351         10.3   
                $\chi_{c0}(1P)$  29.6    -19.2    22     -14.3         24.5   
                $\chi_{c1}(1P)$  28.3    -32.7    45       -52         36.8   
                $\chi_{c2}(1P)$    27     74.6    46       127           41   
                $\eta_{c}(2S)$  0.207    -23.5  0.19     -21.5        0.148   
$\chi_{c0}(2P)$ $\psi(2S)$        nan    -34.6    99     -33.3          103   
$\chi_{c1}(2P)$ $\psi(2S)$        nan    -92.6   155     -82.5          174   
$\chi_{c2}(2P)$ $\psi(2S)$        nan      132   150       113          175   
$\chi_{c0}(3P)$ $\psi(2S)$        nan    -3.02   9.1    -0.425         64.7   
$\chi_{c1}(3P)$ $\psi(2S)$        nan    -8.53    74     -3.97          159   
$\chi_{c2}(3P)$ $\psi(2S)$        nan     14.3    76      6.14          178   

                                $SR-\Gamma$  
In              Out                          
$\psi(2S)$      $\eta_{c}(1S)$       -0.462  
                $\chi_{c0}(1P)$       -15.9  
                $\chi_{c1}(1P)$       -42.6  
                $\chi_{c2}(1P)$         113  
                $\eta_{c}(2S)$        -16.8  
$\chi_{c0}(2P)$ $\psi(2S)$            -34.6  
$\chi_{c1}(2P)$ $\psi(2S)$            -92.6  
$\chi_{c2}(2P)$ $\psi(2S)$              132  
$\chi_{c0}(3P)$ $\psi(2S)$            -3.02  
$\chi_{c1}(3P)$ $\psi(2S)$            -8.53  
$\chi_{c2}(3P)$ $\psi(2S)$             14.3

'Approximation: ELW'

,$SR$
$SR-PDG$,32.3
$SR-Deng$,38.1
$SR-\Gamma$,69.7


PDG $SR-PDG$  Deng $SR-Deng$  ELW-$\Gamma$  \
In              Out                                                            
$\psi(2S)$      $\eta_{c}(1S)$   1.01  -0.0453   7.8    -0.351          4.93   
                $\chi_{c0}(1P)$  29.6    -19.2    22     -14.3          30.3   
                $\chi_{c1}(1P)$  28.3    -32.7    45       -52          39.9   
                $\chi_{c2}(1P)$    27     74.6    46       127          43.2   
                $\eta_{c}(2S)$  0.207    -23.5  0.19     -21.5         0.149   
$\chi_{c0}(2P)$ $\psi(2S)$        nan    -39.4    99     -33.3           117   
$\chi_{c1}(2P)$ $\psi(2S)$        nan     -123   155     -82.5           231   
$\chi_{c2}(2P)$ $\psi(2S)$        nan      191   150       113           254   
$\chi_{c0}(3P)$ $\psi(2S)$        nan -0.00193   9.1    -0.425        0.0413   
$\chi_{c1}(3P)$ $\psi(2S)$        nan    -2.23    74     -3.97          41.7   
$\chi_{c2}(3P)$ $\psi(2S)$        nan      6.3    76      6.14            78   

                                $SR-\Gamma$  
In              Out                          
$\psi(2S)$      $\eta_{c}(1S)$       -0.222  
                $\chi_{c0}(1P)$       -19.6  
                $\chi_{c1}(1P)$       -46.1  
                $\chi_{c2}(1P)$         120  
                $\eta_{c}(2S)$        -16.8  
$\chi_{c0}(2P)$ $\psi(2S)$            -39.4  
$\chi_{c1}(2P)$ $\psi(2S)$             -123  
$\chi_{c2}(2P)$ $\psi(2S)$              191  
$\chi_{c0}(3P)$ $\psi(2S)$         -0.00193  
$\chi_{c1}(3P)$ $\psi(2S)$            -2.23  
$\chi_{c2}(3P)$ $\psi(2S)$              6.3

In [222]:
reportSumrules("psi_2S", "lin", ("c-lin",))

'Approximation: tot'

,$SR$
$SR-PDG$,31.7
$SR-Deng$,41.8
$SR-\Gamma$,69


PDG $SR-PDG$  Deng $SR-Deng$  $\Gamma$  \
In              Out                                                        
$\psi(2S)$      $\eta_{c}(1S)$   1.01  -0.0453  8.08    -0.364      9.63   
                $\chi_{c0}(1P)$  29.6    -19.2    22     -14.3        21   
                $\chi_{c1}(1P)$  28.3    -34.5    42     -51.2      35.9   
                $\chi_{c2}(1P)$    27       69    38      97.3      40.9   
                $\eta_{c}(2S)$  0.207    -23.5  0.19     -21.5     0.158   
$\chi_{c0}(2P)$ $\psi(2S)$        nan    -47.3   121     -40.8       141   
$\chi_{c1}(2P)$ $\psi(2S)$        nan    -80.2   139     -50.9       219   
$\chi_{c2}(2P)$ $\psi(2S)$        nan      164   135       121       183   
$\chi_{c0}(3P)$ $\psi(2S)$        nan    -3.74    17    -0.493       129   
$\chi_{c1}(3P)$ $\psi(2S)$        nan    -8.18    94      -2.8       275   
$\chi_{c2}(3P)$ $\psi(2S)$        nan     15.2    97      6.04       244   

                                $SR-\Gamma$  
In              Out                          
$\psi(2S)$      $\eta_{c}(1S)$       -0.433  
                $\chi_{c0}(1P)$       -13.6  
                $\chi_{c1}(1P)$       -43.7  
                $\chi_{c2}(1P)$         105  
                $\eta_{c}(2S)$        -17.9  
$\chi_{c0}(2P)$ $\psi(2S)$            -47.3  
$\chi_{c1}(2P)$ $\psi(2S)$            -80.2  
$\chi_{c2}(2P)$ $\psi(2S)$              164  
$\chi_{c0}(3P)$ $\psi(2S)$            -3.74  
$\chi_{c1}(3P)$ $\psi(2S)$            -8.18  
$\chi_{c2}(3P)$ $\psi(2S)$             15.2

'Approximation: E1'

,$SR$
$SR-PDG$,32.5
$SR-Deng$,27
$SR-\Gamma$,63.4


PDG $SR-PDG$  Deng $SR-Deng$  E1-$\Gamma$  \
In              Out                                                           
$\psi(2S)$      $\eta_{c}(1S)$   1.01  -0.0453  8.08    -0.364         9.63   
                $\chi_{c0}(1P)$  29.6    -19.2    22     -14.3         25.3   
                $\chi_{c1}(1P)$  28.3    -32.7    42     -48.6         37.8   
                $\chi_{c2}(1P)$    27     74.6    38       105         39.3   
                $\eta_{c}(2S)$  0.207    -23.5  0.19     -21.5        0.158   
$\chi_{c0}(2P)$ $\psi(2S)$        nan    -41.8   121     -40.8          124   
$\chi_{c1}(2P)$ $\psi(2S)$        nan    -80.2   139     -55.7          200   
$\chi_{c2}(2P)$ $\psi(2S)$        nan      153   135       102          203   
$\chi_{c0}(3P)$ $\psi(2S)$        nan    -3.32    17    -0.493          115   
$\chi_{c1}(3P)$ $\psi(2S)$        nan    -8.18    94     -3.14          244   
$\chi_{c2}(3P)$ $\psi(2S)$        nan     13.8    97      4.81          279   

                                $SR-\Gamma$  
In              Out                          
$\psi(2S)$      $\eta_{c}(1S)$       -0.433  
                $\chi_{c0}(1P)$       -16.4  
                $\chi_{c1}(1P)$       -43.7  
                $\chi_{c2}(1P)$         109  
                $\eta_{c}(2S)$        -17.9  
$\chi_{c0}(2P)$ $\psi(2S)$            -41.8  
$\chi_{c1}(2P)$ $\psi(2S)$            -80.2  
$\chi_{c2}(2P)$ $\psi(2S)$              153  
$\chi_{c0}(3P)$ $\psi(2S)$            -3.32  
$\chi_{c1}(3P)$ $\psi(2S)$            -8.18  
$\chi_{c2}(3P)$ $\psi(2S)$             13.8

'Approximation: ELW'

,$SR$
$SR-PDG$,74.8
$SR-Deng$,27
$SR-\Gamma$,104


PDG $SR-PDG$  Deng $SR-Deng$  ELW-$\Gamma$  \
In              Out                                                            
$\psi(2S)$      $\eta_{c}(1S)$   1.01  -0.0453  8.08    -0.364           4.8   
                $\chi_{c0}(1P)$  29.6    -19.2    22     -14.3          30.7   
                $\chi_{c1}(1P)$  28.3    -32.7    42     -48.6          40.9   
                $\chi_{c2}(1P)$    27     74.6    38       105          41.2   
                $\eta_{c}(2S)$  0.207    -23.5  0.19     -21.5         0.158   
$\chi_{c0}(2P)$ $\psi(2S)$        nan    -48.2   121     -40.8           143   
$\chi_{c1}(2P)$ $\psi(2S)$        nan     -108   139     -55.7           269   
$\chi_{c2}(2P)$ $\psi(2S)$        nan      228   135       102           303   
$\chi_{c0}(3P)$ $\psi(2S)$        nan   -0.014    17    -0.493         0.484   
$\chi_{c1}(3P)$ $\psi(2S)$        nan     -1.3    94     -3.14          38.8   
$\chi_{c2}(3P)$ $\psi(2S)$        nan     4.44    97      4.81          89.6   

                                $SR-\Gamma$  
In              Out                          
$\psi(2S)$      $\eta_{c}(1S)$       -0.216  
                $\chi_{c0}(1P)$       -19.9  
                $\chi_{c1}(1P)$       -47.3  
                $\chi_{c2}(1P)$         114  
                $\eta_{c}(2S)$        -17.9  
$\chi_{c0}(2P)$ $\psi(2S)$            -48.2  
$\chi_{c1}(2P)$ $\psi(2S)$             -108  
$\chi_{c2}(2P)$ $\psi(2S)$              228  
$\chi_{c0}(3P)$ $\psi(2S)$           -0.014  
$\chi_{c1}(3P)$ $\psi(2S)$             -1.3  
$\chi_{c2}(3P)$ $\psi(2S)$             4.44

### $\psi(3S)$

In [223]:
reportSumrules("psi_3S", "scr", ("c-scr",))

'Approximation: tot'

,$SR$
$SR-PDG$,-91
$SR-Deng$,-91.5
$SR-\Gamma$,-91


PDG  $SR-PDG$  Deng  $SR-Deng$  $\Gamma$  \
In         Out                                                         
$\psi(3S)$ $\eta_{c}(1S)$   nan    -0.143  6.76     -0.101      9.62   
           $\chi_{c0}(1P)$  nan     -1.06   6.7     -0.403      17.6   
           $\chi_{c1}(1P)$  nan     -1.68   6.7     -0.345      32.6   
           $\chi_{c2}(1P)$  nan      2.02   2.5      0.143      35.5   
           $\eta_{c}(2S)$   nan    -0.418  1.78     -0.371      2.01   
           $\chi_{c0}(2P)$  nan      -139    27       -179      20.9   
           $\chi_{c1}(2P)$  nan      -124    67       -210      39.4   
           $\chi_{c2}(2P)$  nan       181    82        322      46.2   
           $\eta_{c}(3S)$   nan     -8.78 0.088      -22.5    0.0344   

                            $SR-\Gamma$  
In         Out                           
$\psi(3S)$ $\eta_{c}(1S)$        -0.143  
           $\chi_{c0}(1P)$        -1.06  
           $\chi_{c1}(1P)$        -1.68  
           $\chi_{c2}(1P)$         2.02  
           $\eta_{c}(2S)$        -0.418  
           $\chi_{c0}(2P)$         -139  
           $\chi_{c1}(2P)$         -124  
           $\chi_{c2}(2P)$          181  
           $\eta_{c}(3S)$         -8.78

'Approximation: E1'

,$SR$
$SR-PDG$,-107
$SR-Deng$,-63
$SR-\Gamma$,-107


PDG  $SR-PDG$  Deng  $SR-Deng$  E1-$\Gamma$  \
In         Out                                                            
$\psi(3S)$ $\eta_{c}(1S)$   nan    -0.143  6.76     -0.101         9.62   
           $\chi_{c0}(1P)$  nan     -1.43   6.7     -0.403         23.9   
           $\chi_{c1}(1P)$  nan     -1.68   6.7      -0.32         35.2   
           $\chi_{c2}(1P)$  nan      2.09   2.5      0.153         34.1   
           $\eta_{c}(2S)$   nan    -0.418  1.78     -0.371         2.01   
           $\chi_{c0}(2P)$  nan      -159    27       -179           24   
           $\chi_{c1}(2P)$  nan      -124    67       -202           41   
           $\chi_{c2}(2P)$  nan       186    82        342         44.7   
           $\eta_{c}(3S)$   nan     -8.78 0.088      -22.5       0.0344   

                            $SR-\Gamma$  
In         Out                           
$\psi(3S)$ $\eta_{c}(1S)$        -0.143  
           $\chi_{c0}(1P)$        -1.43  
           $\chi_{c1}(1P)$        -1.68  
           $\chi_{c2}(1P)$         2.09  
           $\eta_{c}(2S)$        -0.418  
           $\chi_{c0}(2P)$         -159  
           $\chi_{c1}(2P)$         -124  
           $\chi_{c2}(2P)$          186  
           $\eta_{c}(3S)$         -8.78

'Approximation: ELW'

,$SR$
$SR-PDG$,-156
$SR-Deng$,-63
$SR-\Gamma$,-156


PDG  $SR-PDG$  Deng  $SR-Deng$  ELW-$\Gamma$  \
In         Out                                                             
$\psi(3S)$ $\eta_{c}(1S)$   nan   -0.0566  6.76     -0.101           3.8   
           $\chi_{c0}(1P)$  nan    -0.465   6.7     -0.403          7.73   
           $\chi_{c1}(1P)$  nan   -0.0322   6.7      -0.32         0.675   
           $\chi_{c2}(1P)$  nan     0.118   2.5      0.153          1.92   
           $\eta_{c}(2S)$   nan    -0.149  1.78     -0.371         0.714   
           $\chi_{c0}(2P)$  nan      -208    27       -179          31.4   
           $\chi_{c1}(2P)$  nan      -138    67       -202          45.7   
           $\chi_{c2}(2P)$  nan       199    82        342          47.9   
           $\eta_{c}(3S)$   nan      -8.8 0.088      -22.5        0.0345   

                            $SR-\Gamma$  
In         Out                           
$\psi(3S)$ $\eta_{c}(1S)$       -0.0566  
           $\chi_{c0}(1P)$       -0.465  
           $\chi_{c1}(1P)$      -0.0322  
           $\chi_{c2}(1P)$        0.118  
           $\eta_{c}(2S)$        -0.149  
           $\chi_{c0}(2P)$         -208  
           $\chi_{c1}(2P)$         -138  
           $\chi_{c2}(2P)$          199  
           $\eta_{c}(3S)$          -8.8

In [224]:
reportSumrules("psi_3S", "lin", ("c-lin",))

'Approximation: tot'

,$SR$
$SR-PDG$,1.08e+03
$SR-Deng$,1.1e+03
$SR-\Gamma$,1.08e+03


PDG  $SR-PDG$  Deng  $SR-Deng$  $\Gamma$  \
In         Out                                                         
$\psi(3S)$ $\eta_{c}(1S)$   nan    -0.155  6.66    -0.0993      10.4   
           $\chi_{c0}(1P)$  nan     -1.38   5.9     -0.355      22.9   
           $\chi_{c1}(1P)$  nan     -2.16     4     -0.205      42.2   
           $\chi_{c2}(1P)$  nan      2.45  0.25     0.0147      41.7   
           $\eta_{c}(2S)$   nan    -0.584  1.65     -0.344       2.8   
           $\chi_{c0}(2P)$  nan      -178    19       -126      26.8   
           $\chi_{c1}(2P)$  nan      -318    55       -319      54.8   
           $\chi_{c2}(2P)$  nan       252    67        260      64.9   
           $\eta_{c}(3S)$   nan  1.33e+03 0.051   1.29e+03    0.0527   

                            $SR-\Gamma$  
In         Out                           
$\psi(3S)$ $\eta_{c}(1S)$        -0.155  
           $\chi_{c0}(1P)$        -1.38  
           $\chi_{c1}(1P)$        -2.16  
           $\chi_{c2}(1P)$         2.45  
           $\eta_{c}(2S)$        -0.584  
           $\chi_{c0}(2P)$         -178  
           $\chi_{c1}(2P)$         -318  
           $\chi_{c2}(2P)$          252  
           $\eta_{c}(3S)$      1.33e+03

'Approximation: E1'

,$SR$
$SR-PDG$,1.06e+03
$SR-Deng$,1.13e+03
$SR-\Gamma$,1.06e+03


PDG  $SR-PDG$  Deng  $SR-Deng$  E1-$\Gamma$  \
In         Out                                                            
$\psi(3S)$ $\eta_{c}(1S)$   nan    -0.155  6.66    -0.0993         10.4   
           $\chi_{c0}(1P)$  nan     -1.83   5.9     -0.355         30.5   
           $\chi_{c1}(1P)$  nan     -2.16     4     -0.191         45.3   
           $\chi_{c2}(1P)$  nan       2.5  0.25     0.0153         40.8   
           $\eta_{c}(2S)$   nan    -0.584  1.65     -0.344          2.8   
           $\chi_{c0}(2P)$  nan      -207    19       -126         31.2   
           $\chi_{c1}(2P)$  nan      -318    55       -304         57.5   
           $\chi_{c2}(2P)$  nan       260    67        279         62.5   
           $\eta_{c}(3S)$   nan  1.33e+03 0.051   1.29e+03       0.0527   

                            $SR-\Gamma$  
In         Out                           
$\psi(3S)$ $\eta_{c}(1S)$        -0.155  
           $\chi_{c0}(1P)$        -1.83  
           $\chi_{c1}(1P)$        -2.16  
           $\chi_{c2}(1P)$          2.5  
           $\eta_{c}(2S)$        -0.584  
           $\chi_{c0}(2P)$         -207  
           $\chi_{c1}(2P)$         -318  
           $\chi_{c2}(2P)$          260  
           $\eta_{c}(3S)$      1.33e+03

'Approximation: ELW'

,$SR$
$SR-PDG$,973
$SR-Deng$,1.13e+03
$SR-\Gamma$,973


PDG  $SR-PDG$  Deng  $SR-Deng$  ELW-$\Gamma$  \
In         Out                                                             
$\psi(3S)$ $\eta_{c}(1S)$   nan   -0.0629  6.66    -0.0993          4.22   
           $\chi_{c0}(1P)$  nan    -0.439   5.9     -0.355           7.3   
           $\chi_{c1}(1P)$  nan    -0.005     4     -0.191         0.105   
           $\chi_{c2}(1P)$  nan     0.434  0.25     0.0153           7.1   
           $\eta_{c}(2S)$   nan    -0.186  1.65     -0.344         0.894   
           $\chi_{c0}(2P)$  nan      -279    19       -126          41.9   
           $\chi_{c1}(2P)$  nan      -361    55       -304          65.4   
           $\chi_{c2}(2P)$  nan       282    67        279          67.7   
           $\eta_{c}(3S)$   nan  1.33e+03 0.051   1.29e+03        0.0528   

                            $SR-\Gamma$  
In         Out                           
$\psi(3S)$ $\eta_{c}(1S)$       -0.0629  
           $\chi_{c0}(1P)$       -0.439  
           $\chi_{c1}(1P)$       -0.005  
           $\chi_{c2}(1P)$        0.434  
           $\eta_{c}(2S)$        -0.186  
           $\chi_{c0}(2P)$         -279  
           $\chi_{c1}(2P)$         -361  
           $\chi_{c2}(2P)$          282  
           $\eta_{c}(3S)$      1.33e+03

In [225]:
reportSumrules("psi_1_1D", "scr", ("c-scr",), sigma=6)

'Approximation: tot'

,$SR$
$SR-PDG$,-103
$SR-Deng$,-151
$SR-\Gamma$,-157


PDG $SR-PDG$  Deng $SR-Deng$  $\Gamma$  \
In              Out                                                       
$\psi_{1}(1D)$  $\eta_{c}(1S)$   nan  -0.0137   nan   -0.0137     0.427   
                $\chi_{c0}(1P)$  190    -53.2   261     -72.9       288   
                $\chi_{c1}(1P)$ 67.5    -21.2   135     -42.4       152   
                $\chi_{c2}(1P)$  nan     7.04   8.1      5.73      9.94   
                $\eta_{c}(2S)$   nan -0.00097   nan  -0.00097  0.000214   
$\chi_{c0}(2P)$ $\psi_{1}(1D)$   nan    -10.2    12     -17.7      6.87   
$\chi_{c1}(2P)$ $\psi_{1}(1D)$   nan    -24.8   9.8     -24.6      9.88   
$\chi_{c2}(2P)$ $\psi_{1}(1D)$   nan        1  0.46      0.99     0.466   
$\chi_{c0}(3P)$ $\psi_{1}(1D)$   nan   -0.628  0.39   -0.0343      7.14   
$\chi_{c1}(3P)$ $\psi_{1}(1D)$   nan   -0.954     2    -0.202      9.42   
$\chi_{c2}(3P)$ $\psi_{1}(1D)$   nan  -0.0178  0.79   -0.0116      1.21   

                                $SR-\Gamma$  
In              Out                          
$\psi_{1}(1D)$  $\eta_{c}(1S)$      -0.0137  
                $\chi_{c0}(1P)$       -80.5  
                $\chi_{c1}(1P)$       -47.8  
                $\chi_{c2}(1P)$        7.04  
                $\eta_{c}(2S)$     -0.00097  
$\chi_{c0}(2P)$ $\psi_{1}(1D)$        -10.2  
$\chi_{c1}(2P)$ $\psi_{1}(1D)$        -24.8  
$\chi_{c2}(2P)$ $\psi_{1}(1D)$            1  
$\chi_{c0}(3P)$ $\psi_{1}(1D)$       -0.628  
$\chi_{c1}(3P)$ $\psi_{1}(1D)$       -0.954  
$\chi_{c2}(3P)$ $\psi_{1}(1D)$      -0.0178

'Approximation: E1'

,$SR$
$SR-PDG$,-108
$SR-Deng$,-154
$SR-\Gamma$,-146


PDG  $SR-PDG$  Deng $SR-Deng$  E1-$\Gamma$  \
In              Out                                                           
$\psi_{1}(1D)$  $\eta_{c}(1S)$   nan  -0.00672   nan  -0.00672        0.209   
                $\chi_{c0}(1P)$  190     -53.2   261     -72.9          253   
                $\chi_{c1}(1P)$ 67.5     -22.8   135     -45.5          127   
                $\chi_{c2}(1P)$  nan      4.05   8.1      4.67         7.02   
                $\eta_{c}(2S)$   nan -0.000475   nan -0.000475     0.000105   
$\chi_{c0}(2P)$ $\psi_{1}(1D)$   nan     -10.4    12     -17.7         7.01   
$\chi_{c1}(2P)$ $\psi_{1}(1D)$   nan     -26.1   9.8     -23.7         10.8   
$\chi_{c2}(2P)$ $\psi_{1}(1D)$   nan      1.71  0.46      1.42        0.556   
$\chi_{c0}(3P)$ $\psi_{1}(1D)$   nan    -0.608  0.39   -0.0343         6.91   
$\chi_{c1}(3P)$ $\psi_{1}(1D)$   nan     -1.02     2    -0.189         10.8   
$\chi_{c2}(3P)$ $\psi_{1}(1D)$   nan     0.077  0.79      0.11        0.554   

                                $SR-\Gamma$  
In              Out                          
$\psi_{1}(1D)$  $\eta_{c}(1S)$     -0.00672  
                $\chi_{c0}(1P)$       -70.8  
                $\chi_{c1}(1P)$       -42.8  
                $\chi_{c2}(1P)$        4.05  
                $\eta_{c}(2S)$    -0.000475  
$\chi_{c0}(2P)$ $\psi_{1}(1D)$        -10.4  
$\chi_{c1}(2P)$ $\psi_{1}(1D)$        -26.1  
$\chi_{c2}(2P)$ $\psi_{1}(1D)$         1.71  
$\chi_{c0}(3P)$ $\psi_{1}(1D)$       -0.608  
$\chi_{c1}(3P)$ $\psi_{1}(1D)$        -1.02  
$\chi_{c2}(3P)$ $\psi_{1}(1D)$        0.077

'Approximation: ELW'

,$SR$
$SR-PDG$,-109
$SR-Deng$,-154
$SR-\Gamma$,-182


PDG $SR-PDG$  Deng $SR-Deng$  ELW-$\Gamma$  \
In              Out                                                           
$\psi_{1}(1D)$  $\eta_{c}(1S)$   nan      NaN   nan       NaN             0   
                $\chi_{c0}(1P)$  190    -53.2   261     -72.9           349   
                $\chi_{c1}(1P)$ 67.5    -22.8   135     -45.5           153   
                $\chi_{c2}(1P)$  nan     4.73   8.1      4.67          8.22   
                $\eta_{c}(2S)$   nan      NaN   nan       NaN             0   
$\chi_{c0}(2P)$ $\psi_{1}(1D)$   nan    -10.5    12     -17.7          7.12   
$\chi_{c1}(2P)$ $\psi_{1}(1D)$   nan    -28.5   9.8     -23.7          11.8   
$\chi_{c2}(2P)$ $\psi_{1}(1D)$   nan     1.96  0.46      1.42         0.638   
$\chi_{c0}(3P)$ $\psi_{1}(1D)$   nan   -0.746  0.39   -0.0343          8.48   
$\chi_{c1}(3P)$ $\psi_{1}(1D)$   nan  -0.0335     2    -0.189         0.355   
$\chi_{c2}(3P)$ $\psi_{1}(1D)$   nan     0.01  0.79      0.11        0.0719   

                                $SR-\Gamma$  
In              Out                          
$\psi_{1}(1D)$  $\eta_{c}(1S)$          NaN  
                $\chi_{c0}(1P)$       -97.4  
                $\chi_{c1}(1P)$       -51.8  
                $\chi_{c2}(1P)$        4.73  
                $\eta_{c}(2S)$          NaN  
$\chi_{c0}(2P)$ $\psi_{1}(1D)$        -10.5  
$\chi_{c1}(2P)$ $\psi_{1}(1D)$        -28.5  
$\chi_{c2}(2P)$ $\psi_{1}(1D)$         1.96  
$\chi_{c0}(3P)$ $\psi_{1}(1D)$       -0.746  
$\chi_{c1}(3P)$ $\psi_{1}(1D)$      -0.0335  
$\chi_{c2}(3P)$ $\psi_{1}(1D)$         0.01

In [226]:
reportSumrules("psi_1_1D", "lin", ("c-lin",))

'Approximation: tot'

,$SR$
$SR-PDG$,-116
$SR-Deng$,-156
$SR-\Gamma$,-168


PDG  $SR-PDG$  Deng $SR-Deng$  $\Gamma$  \
In              Out                                                        
$\psi_{1}(1D)$  $\eta_{c}(1S)$   nan   -0.0117   nan   -0.0117     0.365   
                $\chi_{c0}(1P)$  190     -53.2   272       -76       288   
                $\chi_{c1}(1P)$ 67.5     -21.3   138     -43.5       146   
                $\chi_{c2}(1P)$  nan      6.24   7.1      4.99      8.87   
                $\eta_{c}(2S)$   nan -0.000628   nan -0.000628  0.000138   
$\chi_{c0}(2P)$ $\psi_{1}(1D)$   nan     -24.2    20     -29.6      16.4   
$\chi_{c1}(2P)$ $\psi_{1}(1D)$   nan       -22   7.9     -12.5      13.9   
$\chi_{c2}(2P)$ $\psi_{1}(1D)$   nan      1.13  0.36     0.652     0.626   
$\chi_{c0}(3P)$ $\psi_{1}(1D)$   nan     -1.37  0.27   -0.0132      28.1   
$\chi_{c1}(3P)$ $\psi_{1}(1D)$   nan     -1.06   3.2    -0.184      18.3   
$\chi_{c2}(3P)$ $\psi_{1}(1D)$   nan    -0.023   1.5   -0.0153      2.27   

                                $SR-\Gamma$  
In              Out                          
$\psi_{1}(1D)$  $\eta_{c}(1S)$      -0.0117  
                $\chi_{c0}(1P)$       -80.4  
                $\chi_{c1}(1P)$       -46.1  
                $\chi_{c2}(1P)$        6.24  
                $\eta_{c}(2S)$    -0.000628  
$\chi_{c0}(2P)$ $\psi_{1}(1D)$        -24.2  
$\chi_{c1}(2P)$ $\psi_{1}(1D)$          -22  
$\chi_{c2}(2P)$ $\psi_{1}(1D)$         1.13  
$\chi_{c0}(3P)$ $\psi_{1}(1D)$        -1.37  
$\chi_{c1}(3P)$ $\psi_{1}(1D)$        -1.06  
$\chi_{c2}(3P)$ $\psi_{1}(1D)$       -0.023

'Approximation: E1'

,$SR$
$SR-PDG$,-121
$SR-Deng$,-159
$SR-\Gamma$,-158


PDG  $SR-PDG$  Deng $SR-Deng$  E1-$\Gamma$  \
In              Out                                                           
$\psi_{1}(1D)$  $\eta_{c}(1S)$   nan  -0.00574   nan  -0.00574        0.179   
                $\chi_{c0}(1P)$  190     -53.2   272       -76          255   
                $\chi_{c1}(1P)$ 67.5     -22.8   138     -46.6          123   
                $\chi_{c2}(1P)$  nan      3.71   7.1      4.09         6.44   
                $\eta_{c}(2S)$   nan -0.000308   nan -0.000308     6.78e-05   
$\chi_{c0}(2P)$ $\psi_{1}(1D)$   nan     -24.9    20     -29.6         16.9   
$\chi_{c1}(2P)$ $\psi_{1}(1D)$   nan     -23.4   7.9       -12         15.5   
$\chi_{c2}(2P)$ $\psi_{1}(1D)$   nan      2.32  0.36      1.11        0.753   
$\chi_{c0}(3P)$ $\psi_{1}(1D)$   nan     -1.38  0.27   -0.0132         28.3   
$\chi_{c1}(3P)$ $\psi_{1}(1D)$   nan     -1.13   3.2    -0.171         21.3   
$\chi_{c2}(3P)$ $\psi_{1}(1D)$   nan    0.0784   1.5     0.116         1.02   

                                $SR-\Gamma$  
In              Out                          
$\psi_{1}(1D)$  $\eta_{c}(1S)$     -0.00574  
                $\chi_{c0}(1P)$       -71.3  
                $\chi_{c1}(1P)$       -41.5  
                $\chi_{c2}(1P)$        3.71  
                $\eta_{c}(2S)$    -0.000308  
$\chi_{c0}(2P)$ $\psi_{1}(1D)$        -24.9  
$\chi_{c1}(2P)$ $\psi_{1}(1D)$        -23.4  
$\chi_{c2}(2P)$ $\psi_{1}(1D)$         2.32  
$\chi_{c0}(3P)$ $\psi_{1}(1D)$        -1.38  
$\chi_{c1}(3P)$ $\psi_{1}(1D)$        -1.13  
$\chi_{c2}(3P)$ $\psi_{1}(1D)$       0.0784

'Approximation: ELW'

,$SR$
$SR-PDG$,-121
$SR-Deng$,-159
$SR-\Gamma$,-190


PDG $SR-PDG$  Deng $SR-Deng$  ELW-$\Gamma$  \
In              Out                                                           
$\psi_{1}(1D)$  $\eta_{c}(1S)$   nan      NaN   nan       NaN             0   
                $\chi_{c0}(1P)$  190    -53.2   272       -76           339   
                $\chi_{c1}(1P)$ 67.5    -22.8   138     -46.6           146   
                $\chi_{c2}(1P)$  nan     4.26   7.1      4.09          7.39   
                $\eta_{c}(2S)$   nan      NaN   nan       NaN             0   
$\chi_{c0}(2P)$ $\psi_{1}(1D)$   nan    -25.7    20     -29.6          17.4   
$\chi_{c1}(2P)$ $\psi_{1}(1D)$   nan    -26.4   7.9       -12          17.4   
$\chi_{c2}(2P)$ $\psi_{1}(1D)$   nan     2.79  0.36      1.11         0.908   
$\chi_{c0}(3P)$ $\psi_{1}(1D)$   nan   -0.427  0.27   -0.0132          8.75   
$\chi_{c1}(3P)$ $\psi_{1}(1D)$   nan  -0.0175   3.2    -0.171         0.328   
$\chi_{c2}(3P)$ $\psi_{1}(1D)$   nan  0.00675   1.5     0.116        0.0873   

                                $SR-\Gamma$  
In              Out                          
$\psi_{1}(1D)$  $\eta_{c}(1S)$          NaN  
                $\chi_{c0}(1P)$       -94.8  
                $\chi_{c1}(1P)$       -49.3  
                $\chi_{c2}(1P)$        4.26  
                $\eta_{c}(2S)$          NaN  
$\chi_{c0}(2P)$ $\psi_{1}(1D)$        -25.7  
$\chi_{c1}(2P)$ $\psi_{1}(1D)$        -26.4  
$\chi_{c2}(2P)$ $\psi_{1}(1D)$         2.79  
$\chi_{c0}(3P)$ $\psi_{1}(1D)$       -0.427  
$\chi_{c1}(3P)$ $\psi_{1}(1D)$      -0.0175  
$\chi_{c2}(3P)$ $\psi_{1}(1D)$      0.00675

In [227]:
raise KeyboardInterrupt("Enough for output")

KeyboardInterrupt: Enough for output

# Sandbox

In [ ]:
def norm(pname, outdirname):
    cfg = json.load(open(os.path.join("output", outdirname, "config"), "r"), object_pairs_hook=OrderedDict)
    prefix = outdirname.split(".")[0]
    points = pd.read_csv(os.path.join("../quarkU/output", prefix+"."+cfg["system"], "data", pname), index_col=0)
    return sp.sqrt(sp.integrate.trapz(points["u"]**2, points.index.values))

In [ ]:
def plot(ax, pname, outdirname):
    cfg = json.load(open(os.path.join("output", outdirname, "config"), "r"), object_pairs_hook=OrderedDict)
    prefix = outdirname.split(".")[0]
    points = pd.read_csv(os.path.join("../quarkU/output", prefix+"."+cfg["system"], "data", pname), index_col=0)
    norm = sp.sqrt(sp.integrate.trapz(points["u"]**2, points.index.values))
    f = sp.interpolate.interp1d(points.index.values, points["u"])
    ax.plot(points.index.values*0.19732697, points["u"]**2/norm**2/0.19732697, ".")

In [ ]:
def overlap(pname1, pname2, outdirname, func = lambda r: r):
    cfg = json.load(open(os.path.join("output", outdirname, "config"), "r"), object_pairs_hook=OrderedDict)
    prefix = outdirname.split(".")[0]
    points1 = pd.read_csv(os.path.join("../quarkU/output", prefix+"."+cfg["system"], "data", pname1), index_col=0)
    points2 = pd.read_csv(os.path.join("../quarkU/output", prefix+"."+cfg["system"], "data", pname2), index_col=0)
    f1 = sp.interpolate.interp1d(points1.index.values, points1["u"])
    f2 = sp.interpolate.interp1d(points2.index.values, points2["u"])
    if (points1.index.values[-1] > points2.index.values[-1]):
        xes = points2.index.values
    else:
        xes = points1.index.values
    return sp.integrate.trapz(f1(xes)*f2(xes)*func(xes), xes)/sp.sqrt(sp.integrate.trapz(points1["u"]**2, points1.index.values)*sp.integrate.trapz(points2["u"]**2, points2.index.values))

In [ ]:
def readCSV(outdirname):
    data = pd.read_csv(open(os.path.join("output", outdirname, "data")), index_col=[0,1])
    cfg = json.load(open(os.path.join("output", outdirname, "eigen_config"), "r"), object_pairs_hook=OrderedDict)
    def k(row):
        Mf = 2*cfg[row.name[1]]["eq"]["env"]["mC"]+cfg[row.name[1]]["eq"]["E"]
        return -Mf+sp.sqrt(Mf**2 + 2*Mf*(cfg[row.name[0]]["eq"]["E"] - cfg[row.name[1]]["eq"]["E"]))
    lap1 = data.apply(lambda r: overlap(*r.name, outdirname), axis=1).rename("melLW")
    lapful = data.apply(lambda r: overlap(*r.name, outdirname, lambda x: x*(sp.special.spherical_jn(0, k(r)*x/2) - sp.sqrt(5/2)*sp.special.spherical_jn(2, k(r)*x/2))), axis=1).rename("melE1")
    gamma = data.apply(k, axis=1).rename("$E_\gamma$")
    data = pd.concat([data, lap1, lapful, gamma], axis=1)
    check = ((data["widthE1"]/data["melE1"]**2-data["widthELW"]/data["melLW"]**2)*2/(data["widthE1"]/data["melE1"]**2+data["widthELW"]/data["melLW"]**2)).rename("check")
    data = pd.concat([data, check], axis=1)
    return data

In [ ]:
def assembleData(datapath, paperpath):
    chin = pd.read_csv(paperpath, index_col=[0,1]).rename(columns={"Gtot": "widthPAPER"})
    report = readCSV(datapath)
    report = pd.concat([report, chin], axis=1)
    
    return report

In [ ]:
def relativeQuant(report):
    return pd.DataFrame(index=report.index)\
        .join(\
            (report["widthPAPER"]/report["widthE1"])\
                .rename("PAPER/E1"))\
        .join(\
              (report["widthPAPER"]/report["width"])\
                  .rename("PAPER/width"))\
        .join(\
              (report["widthPAPER"]/report["width_mod"])\
                  .rename("PAPER/width_mod"))\
        .join(\
              (report["widthPAPER"]/report["widthE1_mod"])\
                  .rename("PAPER/widthE1_mod"))\
        .join(\
              (report["widthPAPER"]/report["widthELW_mod"])\
                  .rename("PAPER/widthELW_mod")\
             )

In [ ]:
def experimentalReport(report, particledata, transdata):
    trans = pd.read_csv(transdata, index_col=[0, 1]).add_suffix("_pdg")
    pdata = pd.read_csv(particledata, index_col=0)
    width = trans.apply(lambda r: r["frac_width_pdg"]*pdata.loc[r.name[0], "PDG_width"],axis=1).rename("width_pdg")
    eMix = report["$E_\gamma$"].copy()
    collection = []
    collection.append(report[["width", "widthE1", "widthELW", "$E_\gamma$", "widthPAPER"]])
    try:
        notnanE = sp.logical_not(sp.isnan(trans["egamma_pdg"]))
        notnanE = [i for i in notnanE[notnanE].index.values for j in report.index.values if i == j]
        eMix[notnanE] = trans["egamma_pdg"][notnanE].copy()
        collection.append(trans[["egamma_pdg"]])
    except KeyError:
        pass
    eMix.rename("E", inplace=True)
    
    MMix = pdata["PDG"].copy()
    nanM = sp.isnan(MMix)
    nanM = nanM[nanM].index.values
    MMix[nanM] = pdata["SP"][nanM].copy()
    MMix /= 1000
    
    expfactor = eMix.index.map(\
              lambda row: eMix[row]*MMix[row[0]]/(eMix[row] + MMix[row[0]])*(MMix[row[0]] - MMix[row[1]])**2\
        )
    expfactor = pd.DataFrame(expfactor.values, index=eMix.index)
    
    width_mod = (expfactor[0]*report["R_width"].copy()).rename("width_mod")
    widthE1_mod = (expfactor[0]*report["R_widthE1"].copy()).rename("widthE1_mod")
    widthELW_mod = (expfactor[0]*report["R_widthELW"].copy()).rename("widthELW_mod")
    collection += [width_mod, widthE1_mod, widthELW_mod]
    
    return pd.concat(collection, join="inner", axis=1).join(width)

In [ ]:
def doReport(datapath, particledata, paperpath, transdata):
    data = assembleData(datapath, paperpath)
    exper = experimentalReport(data, particledata, transdata)
    rel = (relativeQuant(exper)-1.)*100
    rel.rename(columns={i: i+"(%)" for i in rel.columns}, inplace=True)
    
    labels = ["width", "widthE1", "widthELW", "$E_\gamma$", "widthPAPER"]
    for labe in ["Ge1", "Gm1"]:
        if labe in data.columns:
            labels.append(labe)
    data[labels] *= 10**6
    data.rename(columns=dict(zip(labels, ["%s(KeV)"%i for i in labels])), inplace=True)
    
    labels = ["melLW", "melE1"]
    data[labels] *= 10**(-6)
    data.rename(columns=dict(zip(labels, ["%s($KeV^{-1}$)"%i for i in labels])), inplace=True)
    
    labels = ["R_width", "R_widthE1", "R_widthELW"]
    data[labels] *= 10**(-12)
    data.rename(columns=dict(zip(labels, ["%s($KeV^{-2}$)"%i for i in labels])), inplace=True)
    
    labels = ["width", "widthE1", "widthELW", "width_mod", "widthE1_mod", "widthELW_mod", "$E_\gamma$", "widthPAPER", "width_pdg"]
    for labe in ["egamma_pdg"]:
        if labe in exper.columns:
            labels.append(labe)
    exper[labels] *= 10**6
    exper.rename(columns=dict(zip(labels, ["%s(KeV)"%i for i in labels])), inplace=True)
    
    display(data)
    display(exper)
    display(rel)

In [ ]:
def readTechs(particle, cfgnames, prefix):
    stack = []
    for cfgname in cfgnames:
        stack.append(pd.read_csv("output/{}.{}/melsq".format(prefix, cfgname), header=0, index_col=[0,1], names=["In", "Out", "Tot", "E1", "ELW"]))
    res = pd.concat(stack)
    res = res.query("In == '{0}' | Out == '{0}'".format(particle))
    res.index = pd.Index([tuple("${}$".format(code2name(n)) for n in p) for p in res.index.values], names=["In", "Out"])
    return res

In [ ]:
readTechs("chi_b2_1P", ("1P1D2D.b-scr", "nSnP.b-scr"), "scr")

In [ ]:
doReport("scr.E1.1P1D2D.b-scr", "../data/bottomonia.csv", "../data/b-scr.deng2017-1P1D2D.csv", "../data/bottom-trans-PDG.csv")

In [ ]:
doReport("scr.E1.nSnP.b-scr", "../data/bottomonia.csv", "../data/b-scr.deng2017-nSnP.csv", "../data/bottom-trans-PDG.csv")

In [ ]:
doReport("scr.c-scr","../data/charmonia.csv", "../data/c-scr.deng2017.csv", "../data/charm-trans-PDG.csv")

In [ ]:
doReport("lin.c-lin","../data/charmonia.csv", "../data/c-lin.deng2017.csv", "../data/charm-trans-PDG.csv")